<a href="https://colab.research.google.com/github/PreyPython123/Master-V24-Semiveiledet-Regresjon/blob/Collagen-Pradeep/Bioco_Collagen_Klassiske_Superveiledet_Regresjonsmetoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importering, lasting og formattering

Lasting av nødvendige bibliotek og pakker

In [1]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 11.2 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


Importering av nødvendig bibliotek og pakker

In [3]:
import pandas as pd
import numpy as np
import copy
import math

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold
import sklearn.model_selection
from statistics import mean

from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR


import optuna
from optuna.visualization import plot_optimization_history

# Utelukker utskriving av logg for hyperoptimalisering
# optuna.logging.set_verbosity(optuna.logging.WARNING)

Evalueringsmetrikker

In [4]:
from sklearn.metrics import (mean_absolute_error,
                             mean_squared_error,
                             mean_absolute_percentage_error,
                             r2_score)

# Lager en funksjon for RMSE
def rmse(y_faktisk, y_predikert):
  return np.sqrt(mean_squared_error(y_faktisk, y_predikert))

# Egendefinerte moduler

In [5]:
!pwd
! cp /content/drive/MyDrive/MasterV24/GitHub/oppdeling_trening_testsett.py /content/oppdeling_trening_testsett.py

/content


In [6]:
from oppdeling_trening_testsett import trening_testsett_oppdeling_enzym
from oppdeling_trening_testsett import trening_testsett_oppdeling_dag
from oppdeling_trening_testsett import trening_validering_oppdeling_dag

In [7]:
def evaluering_beregning(y_test, y_test_prediksjon, y_trening, y_trening_prediksjon):
  rmse_test = rmse(y_test, y_test_prediksjon)
  r2_test = r2_score(y_test, y_test_prediksjon)
  mae_test = mean_absolute_error(y_test, y_test_prediksjon)
  mape_test = mean_absolute_percentage_error(y_test, y_test_prediksjon)

  rmse_trening = rmse(y_trening, y_trening_prediksjon)
  r2_trening = r2_score(y_trening, y_trening_prediksjon)
  mae_trening = mean_absolute_error(y_trening, y_trening_prediksjon)
  mape_trening = mean_absolute_percentage_error(y_trening, y_trening_prediksjon)

  return rmse_test, r2_test, mae_test, mape_test, rmse_trening, r2_trening, mae_trening, mape_trening

In [8]:
def evaluering_resultater(detailed_objective):
  rmse_test = detailed_objective(study.best_trial)[0]
  r2_test = detailed_objective(study.best_trial)[1]
  mae_test = detailed_objective(study.best_trial)[2]
  mape_test = detailed_objective(study.best_trial)[3]

  rmse_trening = detailed_objective(study.best_trial)[4]
  r2_trening = detailed_objective(study.best_trial)[5]
  mae_trening = detailed_objective(study.best_trial)[6]
  mape_trening = detailed_objective(study.best_trial)[7]

  return rmse_test, r2_test, mae_test, mape_test, rmse_trening, r2_trening, mae_trening, mape_trening

In [9]:
def rmse_validering_StratifiedKFold(modell, datasett, respons, n_splitt):

  skf = StratifiedKFold(n_splits=n_splitt)

  rmse_validering_resultat = []

  for trening, validering in skf.split(datasett, respons):
    X_trening_fold = (datasett.iloc[trening, :]).iloc[:, :-1]
    X_validering_fold = (datasett.iloc[validering, :]).iloc[:, :-1]

    y_trening_fold = (datasett.iloc[trening, :]).iloc[:, -1]
    y_validering_fold = (datasett.iloc[validering, :]).iloc[:, -1]

    modell.fit(X_trening_fold, y_trening_fold)

    y_validering_prediksjon = modell.predict(X_validering_fold)

    rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
    rmse_validering_resultat.append(rmse_validering)

  return mean(rmse_validering)

# Importering av relevant data

In [10]:
# Velger første kolonne med dato og tid som index
collagen_data = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/collagen_data.csv',
                            header=0,
                            sep=',',
                            index_col=0)

# Normal produksjon
collagen_data_normal = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/collagen_data_normal.csv',
                                   header=0,
                                   sep=',',
                                   index_col=0)

collagen_data_design = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/collagen_data_design.csv',
                                   header=0,
                                   sep=',',
                                   index_col=0)

# Importerer rådata
rå_data = pd.read_csv('/content/drive/MyDrive/MasterV24/BiocoData.csv',
                      header=0,
                      sep=';',
                      index_col=0)

# Formatterer index til riktig format og datatype
collagen_data.index = pd.to_datetime(collagen_data.index,
                                     format='%Y-%m-%d %H:%M:%S')

collagen_data_normal.index = pd.to_datetime(collagen_data_normal.index,
                                            format='%Y-%m-%d %H:%M:%S')

collagen_data_design.index = pd.to_datetime(collagen_data_design.index,
                                            format='%Y-%m-%d %H:%M:%S')

rå_data.index = pd.to_datetime(rå_data.index,
                               format='%d-%m-%Y %H:%M:%S.%f')

Tilfeldighetsfrø

In [11]:
#Tilfeldighetsfrø
random_seed = 123

# Beskrivelse av datasett

In [12]:
collagen_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 29136 entries, 2022-10-31 17:37:00 to 2023-06-14 01:06:00
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   EnzymeType_A1  29136 non-null  int64  
 1   EnzymeType_A2  29136 non-null  int64  
 2   EnzymeType_B   29136 non-null  int64  
 3   EnzymeType_C   29136 non-null  int64  
 4   EnzymeType_D   29136 non-null  int64  
 5   EnzymeType_E   29136 non-null  int64  
 6   RawMatPercent  29136 non-null  float64
 7   NIRfat         29136 non-null  float64
 8   NIRash         29136 non-null  float64
 9   NIRwater       29136 non-null  float64
 10  TT08           29136 non-null  float64
 11  TT20           29136 non-null  float64
 12  TT12           29136 non-null  float64
 13  Collagen       89 non-null     float64
dtypes: float64(8), int64(6)
memory usage: 3.3 MB


In [13]:
collagen_data_normal.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 18568 entries, 2022-11-02 00:00:00 to 2023-06-14 01:06:00
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   EnzymeType_A1  18568 non-null  int64  
 1   EnzymeType_A2  18568 non-null  int64  
 2   EnzymeType_B   18568 non-null  int64  
 3   EnzymeType_C   18568 non-null  int64  
 4   EnzymeType_D   18568 non-null  int64  
 5   EnzymeType_E   18568 non-null  int64  
 6   RawMatPercent  18568 non-null  float64
 7   NIRfat         18568 non-null  float64
 8   NIRash         18568 non-null  float64
 9   NIRwater       18568 non-null  float64
 10  TT08           18568 non-null  float64
 11  TT20           18568 non-null  float64
 12  TT12           18568 non-null  float64
 13  Collagen       31 non-null     float64
dtypes: float64(8), int64(6)
memory usage: 2.1 MB


In [14]:
collagen_data_design.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 10568 entries, 2022-10-31 17:37:00 to 2023-06-13 23:59:00
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   EnzymeType_A1  10568 non-null  int64  
 1   EnzymeType_A2  10568 non-null  int64  
 2   EnzymeType_B   10568 non-null  int64  
 3   EnzymeType_C   10568 non-null  int64  
 4   EnzymeType_D   10568 non-null  int64  
 5   EnzymeType_E   10568 non-null  int64  
 6   RawMatPercent  10568 non-null  float64
 7   NIRfat         10568 non-null  float64
 8   NIRash         10568 non-null  float64
 9   NIRwater       10568 non-null  float64
 10  TT08           10568 non-null  float64
 11  TT20           10568 non-null  float64
 12  TT12           10568 non-null  float64
 13  Collagen       58 non-null     float64
dtypes: float64(8), int64(6)
memory usage: 1.2 MB


# NIR målinger inkludert

## Hele datasett

## Alternativ 1: Et testsett. Fordelt på enzymtype (ikke tatt hensyn til kontinuitet)

In [16]:
# Definerer testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
collagen_trening_markert, collagen_enzymtyper_markert, collagen_test, _, _, _, _\
=trening_testsett_oppdeling_enzym(rå_data,
                                  collagen_data,
                                  test_andel=test_andel)

# Deler datasettene i verdier for forklaringsvariabler og responsvariabler
X_trening_markert = collagen_trening_markert.iloc[:, :-1]
X_test = collagen_test.iloc[:, :-1]
y_trening_markert = collagen_trening_markert.iloc[:, -1]
y_test = collagen_test.iloc[:, -1]

**RandomForestRegressor**

Hyperparamter optimalisering med Optuna

In [17]:
def objective(trial):

  parametere = {
      'n_estimators': trial.suggest_int('n_estimators', 100, 200),
      'max_depth': trial.suggest_int('max_depth', 1, 10)
  }
  rfr_pipeline = Pipeline([
      ('skalerer', StandardScaler()),
      ('modell', RandomForestRegressor(**parametere,
                                       random_state=random_seed))
  ])

  # Manuell Kryssvalidering med Kfold og optimalisering ift. gjennomsnittlig rmse
  splitt = 2
  gjentagelser = 2

  rskf = RepeatedStratifiedKFold(n_splits = splitt,
                                 n_repeats=gjentagelser,
                                 random_state=random_seed)

  rmse_validering_resultat = []

  for train, test in rskf.split(collagen_trening_markert,
                                collagen_enzymtyper_markert):

    X_trening_fold = (collagen_trening_markert.iloc[train, :]).iloc[:, :-1]
    X_validering_fold = (collagen_trening_markert.iloc[test, :]).iloc[:, :-1]

    y_trening_fold = (collagen_trening_markert.iloc[train, :]).iloc[:, -1]
    y_validering_fold = (collagen_trening_markert.iloc[test, :]).iloc[:, -1]

    rfr_pipeline.fit(X_trening_fold, y_trening_fold)

    y_validering_prediksjon = rfr_pipeline.predict(X_validering_fold)
    rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
    rmse_validering_resultat.append(rmse_validering)

  rmse_validering_snitt = mean(rmse_validering_resultat)

  return rmse_validering_snitt

if __name__ == "__main__":
  study = optuna.create_study(direction='minimize')
  study.optimize(objective, n_trials=100)

plot_optimization_history(study)

[I 2024-03-11 19:19:45,845] A new study created in memory with name: no-name-5c525c83-93cc-4106-85f1-3a859fbf990f
[I 2024-03-11 19:19:46,794] Trial 0 finished with value: 3.7461175671461215 and parameters: {'n_estimators': 106, 'max_depth': 9}. Best is trial 0 with value: 3.7461175671461215.
[I 2024-03-11 19:19:48,354] Trial 1 finished with value: 3.9512473243698727 and parameters: {'n_estimators': 191, 'max_depth': 1}. Best is trial 0 with value: 3.7461175671461215.
[I 2024-03-11 19:19:50,043] Trial 2 finished with value: 3.6999718182561425 and parameters: {'n_estimators': 198, 'max_depth': 9}. Best is trial 2 with value: 3.6999718182561425.
[I 2024-03-11 19:19:50,722] Trial 3 finished with value: 3.7056460342892374 and parameters: {'n_estimators': 127, 'max_depth': 2}. Best is trial 2 with value: 3.6999718182561425.
[I 2024-03-11 19:19:51,367] Trial 4 finished with value: 3.7435774844132306 and parameters: {'n_estimators': 117, 'max_depth': 6}. Best is trial 2 with value: 3.699971818

Evaluering av beste modell

In [18]:
collagen_rfr_resultater_enzym = pd.DataFrame(columns = ["Test andel",
                                                        "RMSE test",
                                                        "R2 test",
                                                        "MAE test",
                                                        "MAPE test",
                                                        "RMSE trening",
                                                        "R2 trening",
                                                        "MAE trening",
                                                        "MAPE trening",
                                                        "Beste parametere"])

# Bruker beste paramtere for studiet for å lage modell
beste_parametere = study.best_params

rfr_pipeline = Pipeline([
    ('skalerer', StandardScaler()),
    ('modell', RandomForestRegressor(**beste_parametere,
                                     random_state=random_seed))
    ])

# Trener modell for videre evaluering
rfr_pipeline.fit(X_trening_markert, y_trening_markert)
y_test_prediksjon = rfr_pipeline.predict(X_test)
y_trening_prediksjon = rfr_pipeline.predict(X_trening_markert)

# Henter ut evalueringsresulateter av beste modell fra undersøkelse
rmse_test, r2_test, mae_test, mape_test,\
rmse_trening, r2_trening, mae_trening, mape_trening\
= evaluering_beregning(y_test,
                       y_test_prediksjon,
                       y_trening_markert,
                       y_trening_prediksjon)


# Legger resulater til i datasett
resultater = [test_andel,
              rmse_test, r2_test, mae_test, mape_test,
              rmse_trening, r2_trening, mae_trening, mape_trening,
              beste_parametere]

collagen_rfr_resultater_enzym.loc[len(collagen_rfr_resultater_enzym)] = resultater

In [19]:
collagen_rfr_resultater_enzym

,Test andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,0.2,4.311254,0.465149,3.625746,0.190874,2.719353,0.708171,2.158831,0.105589,"{'n_estimators': 191, 'max_depth': 2}"


## Alternativ 1: Flere testandeler

**RandomForestRegressor**

In [22]:
collagen_rfr_resultater_enzym = pd.DataFrame(columns = ["Test andel",
                                                        "RMSE test",
                                                        "R2 test",
                                                        "MAE test",
                                                        "MAPE test",
                                                        "RMSE trening",
                                                        "R2 trening",
                                                        "MAE trening",
                                                        "MAPE trening",
                                                        "Beste parametere"])

test_andeler = [0.1, 0.2, 0.3, 0.4, 0.5]

for test_andel in test_andeler:

  collagen_trening_markert, collagen_enzymtyper_markert, collagen_test, _, _, _, _\
  =trening_testsett_oppdeling_enzym(rå_data,
                                    collagen_data,
                                    test_andel=test_andel,
                                    random_seed=random_seed)

  X_trening = collagen_trening_markert.iloc[:, :-1]
  X_test = collagen_test.iloc[:, :-1]
  y_trening = collagen_trening_markert.iloc[:, -1]
  y_test = collagen_test.iloc[:, -1]

  def objective(trial):
    parametere = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 200),
        'max_depth': trial.suggest_int('max_depth', 1, 10)
    }
    rfr_pipeline = Pipeline([
        ('skalerer', StandardScaler()),
        ('modell', RandomForestRegressor(**parametere,
                                         random_state=random_seed))
    ])

    # Manuell Kryssvalidering med Kfold og optimalisering ift. gjennomsnittlig rmse
    splits = 2
    skf = StratifiedKFold(n_splits = splits)

    rmse_validering_resultat = []

    for train, test in skf.split(collagen_trening_markert,
                                 collagen_enzymtyper_markert):

      X_trening_fold = (collagen_trening_markert.iloc[train, :]).iloc[:, :-1]
      X_validering_fold = (collagen_trening_markert.iloc[test, :]).iloc[:, :-1]

      y_trening_fold = (collagen_trening_markert.iloc[train, :]).iloc[:, -1]
      y_validering_fold = (collagen_trening_markert.iloc[test, :]).iloc[:, -1]

      rfr_pipeline.fit(X_trening_fold, y_trening_fold)

      y_validering_prediksjon = rfr_pipeline.predict(X_validering_fold)
      rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
      rmse_validering_resultat.append(rmse_validering)

    rmse_validering_snitt = mean(rmse_validering_resultat)

    return rmse_validering_snitt

  if __name__ == "__main__":
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=100)

  # Bruker beste paramtere for studiet for å lage modell
  beste_parametere = study.best_params

  rfr_pipeline = Pipeline([
      ('skalerer', StandardScaler()),
      ('modell', RandomForestRegressor(**beste_parametere,
                                      random_state=random_seed))
      ])

  # Trener modell for videre evaluering
  rfr_pipeline.fit(X_trening_markert, y_trening_markert)
  y_test_prediksjon = rfr_pipeline.predict(X_test)
  y_trening_prediksjon = rfr_pipeline.predict(X_trening_markert)

  # Henter ut evalueringsresulateter av beste modell fra undersøkelse
  rmse_test, r2_test, mae_test, mape_test,\
  rmse_trening, r2_trening, mae_trening, mape_trening\
  = evaluering_beregning(y_test,
                         y_test_prediksjon,
                         y_trening_markert,
                         y_trening_prediksjon)


  # Legger resulater til i datasett
  resultater = [test_andel,
                rmse_test, r2_test, mae_test, mape_test,
                rmse_trening, r2_trening, mae_trening, mape_trening,
                beste_parametere]

  collagen_rfr_resultater_enzym.loc[len(collagen_rfr_resultater_enzym)] = resultater

[I 2024-03-11 19:31:20,759] A new study created in memory with name: no-name-9138cb3b-78bc-4b4c-a850-e85a8655bc55
[I 2024-03-11 19:31:22,941] Trial 0 finished with value: 3.7259815635191758 and parameters: {'n_estimators': 184, 'max_depth': 6}. Best is trial 0 with value: 3.7259815635191758.
[I 2024-03-11 19:31:24,920] Trial 1 finished with value: 3.70545375062886 and parameters: {'n_estimators': 131, 'max_depth': 5}. Best is trial 1 with value: 3.70545375062886.
[I 2024-03-11 19:31:26,165] Trial 2 finished with value: 3.7211457332970608 and parameters: {'n_estimators': 195, 'max_depth': 9}. Best is trial 1 with value: 3.70545375062886.
[I 2024-03-11 19:31:26,516] Trial 3 finished with value: 3.704282716304398 and parameters: {'n_estimators': 124, 'max_depth': 6}. Best is trial 3 with value: 3.704282716304398.
[I 2024-03-11 19:31:26,830] Trial 4 finished with value: 3.7251195060679376 and parameters: {'n_estimators': 110, 'max_depth': 4}. Best is trial 3 with value: 3.704282716304398.


**KNR**

In [ ]:
collagen_knr_resultater_enzym = pd.DataFrame(columns = ["Test andel",
                                                        "RMSE test",
                                                        "R2 test",
                                                        "MAE test",
                                                        "MAPE test",
                                                        "RMSE trening",
                                                        "R2 trening",
                                                        "MAE trening",
                                                        "MAPE trening",
                                                        "Beste parametere"])

test_andeler = [0.1, 0.2, 0.3, 0.4, 0.5]

for test_andel in test_andeler:

  treningssett_markert, enzymtyper_trening_markert, testsett, _, _, _, _\
  =trening_testsett_oppdeling_enzym(rå_data,
                                    collagen_data,
                                    test_andel=test_andel,
                                    random_seed=random_seed)


  X_trening_markert = treningssett_markert.iloc[:, :-1]
  X_test = testsett.iloc[:, :-1]
  y_trening_markert = treningssett_markert.iloc[:, -1]
  y_test = testsett.iloc[:, -1]

  # KNR
  def objective(trial):

    parametere = {
          'n_neighbors': trial.suggest_int('n_neighbors', 1, 20),
          'p': trial.suggest_int('p', 1, 20)
      }
    knr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', KNeighborsRegressor(**parametere))
      ])

    splitt = 2
    gjentagelser = 10

    rskf = RepeatedStratifiedKFold(n_splits = splitt,
                                   n_repeats=gjentagelser,
                                   random_state=random_seed)

    rmse_validering_resultat = []

    for train, test in rskf.split(treningssett_markert,
                                  enzymtyper_trening_markert):

      X_trening_fold = (treningssett_markert.iloc[train, :]).iloc[:, :-1]
      X_validering_fold = (treningssett_markert.iloc[test, :]).iloc[:, :-1]

      y_trening_fold = (treningssett_markert.iloc[train, :]).iloc[:, -1]
      y_validering_fold = (treningssett_markert.iloc[test, :]).iloc[:, -1]

      knr_pipeline.fit(X_trening_fold, y_trening_fold)

      y_validering_prediksjon = knr_pipeline.predict(X_validering_fold)

      rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
      rmse_validering_resultat.append(rmse_validering)

    rmse_validering_snitt = mean(rmse_validering_resultat)

    return rmse_validering_snitt

  if __name__ == "__main__":
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=300)

  # Bruker beste paramtere for studiet for å lage modell
  beste_parametere = study.best_params

  knr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', KNeighborsRegressor(**beste_parametere))
      ])

  # Trener modell for videre evaluering
  knr_pipeline.fit(X_trening_markert, y_trening_markert)
  y_test_prediksjon = knr_pipeline.predict(X_test)
  y_trening_prediksjon = knr_pipeline.predict(X_trening_markert)

  # Henter ut evalueringsresulateter av beste modell fra undersøkelse
  rmse_test, r2_test,\
  mae_test, mape_test,\
  rmse_trening, r2_trening,\
  mae_trening, mape_trening = evaluering_beregning(y_test,
                                                   y_test_prediksjon,
                                                   y_trening_markert,
                                                   y_trening_prediksjon)

  # Legger resulater til i datasett
  resultater = [test_andel,
                rmse_test, r2_test, mae_test, mape_test,
                rmse_trening, r2_trening, mae_trening, mape_trening,
                beste_parametere]

  collagen_knr_resultater_enzym.loc[len(collagen_knr_resultater_enzym)] = resultater

[I 2024-02-29 15:59:41,085] A new study created in memory with name: no-name-3bef9608-2d49-454b-8998-16d3998b98b4
[I 2024-02-29 15:59:41,260] Trial 0 finished with value: 4.546222984836899 and parameters: {'n_neighbors': 18, 'p': 15}. Best is trial 0 with value: 4.546222984836899.
[I 2024-02-29 15:59:41,433] Trial 1 finished with value: 4.273504619153361 and parameters: {'n_neighbors': 11, 'p': 17}. Best is trial 1 with value: 4.273504619153361.
[I 2024-02-29 15:59:41,610] Trial 2 finished with value: 4.234310400238758 and parameters: {'n_neighbors': 10, 'p': 14}. Best is trial 2 with value: 4.234310400238758.
[I 2024-02-29 15:59:41,883] Trial 3 finished with value: 4.486927433058458 and parameters: {'n_neighbors': 20, 'p': 6}. Best is trial 2 with value: 4.234310400238758.
[I 2024-02-29 15:59:42,065] Trial 4 finished with value: 4.210711271094873 and parameters: {'n_neighbors': 8, 'p': 14}. Best is trial 4 with value: 4.210711271094873.
[I 2024-02-29 15:59:42,240] Trial 5 finished wit

In [ ]:
collagen_rfr_resultater_enzym

,Test andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,0.1,4.431145,0.487955,3.908301,0.177555,1.227786,0.940510,0.953507,0.046323,"{'n_estimators': 120, 'max_depth': 7}"
1,0.2,4.003539,0.538774,3.479749,0.176571,1.232411,0.940061,0.955381,0.046436,"{'n_estimators': 119, 'max_depth': 7}"
2,0.3,3.402366,0.651472,2.730617,0.136935,1.227301,0.940557,0.946709,0.046206,"{'n_estimators': 101, 'max_depth': 7}"
3,0.4,3.152136,0.640465,2.545815,0.124999,1.800411,0.872079,1.452228,0.070551,"{'n_estimators': 156, 'max_depth': 4}"
4,0.5,2.815563,0.709283,2.163741,0.105718,1.529586,0.907669,1.239280,0.060315,"{'n_estimators': 101, 'max_depth': 5}"


In [ ]:
collagen_knr_resultater_enzym

,Test andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,0.1,4.232501,0.532835,3.572424,0.170281,3.653331,0.490480,3.007284,0.147001,"{'n_neighbors': 11, 'p': 2}"
1,0.2,4.220846,0.487346,3.451278,0.184210,3.754695,0.443651,3.135042,0.148496,"{'n_neighbors': 10, 'p': 2}"
2,0.3,4.083194,0.498032,3.540823,0.176039,3.844172,0.406610,3.208226,0.152490,"{'n_neighbors': 9, 'p': 2}"
3,0.4,3.867523,0.458752,3.284167,0.152165,3.753440,0.479273,3.071402,0.147141,"{'n_neighbors': 7, 'p': 2}"
4,0.5,3.851310,0.456053,3.244000,0.149294,3.868332,0.454884,3.340942,0.164059,"{'n_neighbors': 7, 'p': 2}"


## Alternativ 1: Flere treningsandeler

In [ ]:
collagen_rfr_resultater_enzym = pd.DataFrame(columns = ["Trenings andel",
                                                        "RMSE test",
                                                        "R2 test",
                                                        "MAE test",
                                                        "MAPE test",
                                                        "RMSE trening",
                                                        "R2 trening",
                                                        "MAE trening",
                                                        "MAPE trening",
                                                        "Beste parametere"])

collagen_knr_resultater_enzym = pd.DataFrame(columns = ["Trenings andel",
                                                         "RMSE test",
                                                         "R2 test",
                                                         "MAE test",
                                                         "MAPE test",
                                                         "RMSE trening",
                                                         "R2 trening",
                                                         "MAE trening",
                                                         "MAPE trening",
                                                         "Beste parametere"])

collagen_svr_resultater_enzym = pd.DataFrame(columns = ["Trenings andel",
                                                         "RMSE test",
                                                         "R2 test",
                                                         "MAE test",
                                                         "MAPE test",
                                                         "RMSE trening",
                                                         "R2 trening",
                                                         "MAE trening",
                                                         "MAPE trening",
                                                         "Beste parametere"])



# Setter fast testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert,test, _, trening_umarkert,\
_, trening = trening_testsett_oppdeling_enzym(rå_data,
                                              collagen_data,
                                              test_andel=test_andel)

# Treningssett
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]


# Treningsandeler
# treningsandeler = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
treningsandeler = [0.5, 0.6]

### **RandomForestRegressor**

In [ ]:
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  X_trening = treningssett.iloc[:, :-1]
  X_trening_markert = treningssett_markert.iloc[:, :-1]
  X_trening_umarkert = treningssett_umarkert.iloc[:, :-1]
  y_trening_markert = treningssett_markert.iloc[:, -1]

  # RandomForestRegressor
  def objective(trial):

    parametere = {
          'n_estimators': trial.suggest_int('n_estimators', 100, 200),
          'max_depth': trial.suggest_int('max_depth', 1, 10)
      }
    rfr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', RandomForestRegressor(**parametere,
                                           random_state=random_seed))
      ])

    splitt = 5
    gjentagelser = 10

    rskf = RepeatedStratifiedKFold(n_splits = splitt,
                                  n_repeats=gjentagelser,
                                  random_state=random_seed)

    rmse_validering_resultat = []

    for train, test in rskf.split(treningssett_markert,
                                  enzymtyper_treningssett_markert):

      X_trening_fold = (treningssett_markert.iloc[train, :]).iloc[:, :-1]
      X_validering_fold = (treningssett_markert.iloc[test, :]).iloc[:, :-1]

      y_trening_fold = (treningssett_markert.iloc[train, :]).iloc[:, -1]
      y_validering_fold = (treningssett_markert.iloc[test, :]).iloc[:, -1]

      rfr_pipeline.fit(X_trening_fold, y_trening_fold)

      y_validering_prediksjon = rfr_pipeline.predict(X_validering_fold)
      rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
      rmse_validering_resultat.append(rmse_validering)

    rmse_validering_snitt = mean(rmse_validering_resultat)

    return rmse_validering_snitt

  if __name__ == "__main__":
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=100)


  # Bruker beste paramtere for studiet for å lage modell
  beste_parametere = study.best_params

  rfr_pipeline = Pipeline([
      ('skalerer', StandardScaler()),
      ('modell', RandomForestRegressor(**beste_parametere,
                                       random_state=random_seed))
  ])

  # Trener modell for videre evaluering
  rfr_pipeline.fit(X_trening_markert, y_trening_markert)
  y_test_prediksjon = rfr_pipeline.predict(X_test)
  y_trening_prediksjon = rfr_pipeline.predict(X_trening_markert)

  # Henter ut evalueringsresulateter av beste modell fra undersøkelse
  rmse_test, r2_test, mae_test, mape_test,\
  rmse_trening, r2_trening, mae_trening, mape_trening\
  = evaluering_beregning(y_test,
                         y_test_prediksjon,
                         y_trening_markert,
                         y_trening_prediksjon)


  # Legger resulater til i datasett
  resultater = [trenings_andel,
                rmse_test, r2_test, mae_test, mape_test,
                rmse_trening, r2_trening, mae_trening, mape_trening,
                beste_parametere]

  collagen_rfr_resultater_enzym.loc[len(collagen_rfr_resultater_enzym)] = resultater

[I 2024-03-14 09:35:07,664] A new study created in memory with name: no-name-79e538bb-a734-4f90-8fce-27042a998946
[I 2024-03-14 09:35:18,360] Trial 0 finished with value: 3.2150190130871548 and parameters: {'n_estimators': 113, 'max_depth': 6}. Best is trial 0 with value: 3.2150190130871548.
[I 2024-03-14 09:35:26,465] Trial 1 finished with value: 3.246628304318582 and parameters: {'n_estimators': 105, 'max_depth': 5}. Best is trial 0 with value: 3.2150190130871548.
[I 2024-03-14 09:35:39,819] Trial 2 finished with value: 3.1789972215274744 and parameters: {'n_estimators': 199, 'max_depth': 2}. Best is trial 2 with value: 3.1789972215274744.
[I 2024-03-14 09:35:47,615] Trial 3 finished with value: 3.2073394492896963 and parameters: {'n_estimators': 140, 'max_depth': 6}. Best is trial 2 with value: 3.1789972215274744.
[I 2024-03-14 09:35:52,601] Trial 4 finished with value: 3.2244694594134646 and parameters: {'n_estimators': 112, 'max_depth': 5}. Best is trial 2 with value: 3.1789972215

In [ ]:
collagen_rfr_resultater_enzym

,Trenings andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,0.3,4.708957,0.361921,3.955126,0.208063,1.577786,0.874150,1.169770,0.050592,"{'n_estimators': 196, 'max_depth': 2}"
1,0.4,4.587188,0.394494,3.651973,0.196802,0.960627,0.940972,0.773301,0.032221,"{'n_estimators': 101, 'max_depth': 5}"
2,0.5,4.548134,0.404761,3.616760,0.192462,1.073443,0.947151,0.847493,0.036098,"{'n_estimators': 161, 'max_depth': 5}"
3,0.6,4.324051,0.461970,3.492317,0.184675,1.245919,0.938645,0.882178,0.042426,"{'n_estimators': 182, 'max_depth': 7}"
4,0.7,4.091217,0.518351,3.409973,0.179577,1.280192,0.929145,1.000691,0.048157,"{'n_estimators': 140, 'max_depth': 6}"
5,0.8,3.652408,0.616130,3.076499,0.157689,1.278056,0.939767,0.990484,0.049713,"{'n_estimators': 166, 'max_depth': 6}"
6,0.9,3.733275,0.598944,3.129767,0.159291,1.122911,0.949992,0.896948,0.043438,"{'n_estimators': 104, 'max_depth': 10}"
7,1.0,4.069398,0.523475,3.567613,0.182993,1.808574,0.870917,1.461230,0.071045,"{'n_estimators': 198, 'max_depth': 4}"


### **KNeighborsRegressor**

In [ ]:
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:
    ubenyttet_andel = (1.0-trenings_andel)
    treningssett_markert, enzymtyper_treningssett_markert,\
    _, _, treningssett_umarkert, _, treningssett\
    = trening_testsett_oppdeling_enzym(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  X_trening = treningssett.iloc[:, :-1]
  X_trening_markert = treningssett_markert.iloc[:, :-1]
  X_trening_umarkert = treningssett_umarkert.iloc[:, :-1]
  y_trening_markert = treningssett_markert.iloc[:, -1]

  # KNR
  def objective(trial):

    parametere = {
          'n_neighbors': trial.suggest_int('n_neighbors', 1, 10),
          'p': trial.suggest_int('p', 1, 10)
      }
    knr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', KNeighborsRegressor(**parametere))
      ])

    splitt = 5
    gjentagelser = 10

    rskf = RepeatedStratifiedKFold(n_splits = splitt,
                                   n_repeats=gjentagelser,
                                   random_state=random_seed)

    rmse_validering_resultat = []

    for train, test in rskf.split(treningssett_markert,
                                  enzymtyper_treningssett_markert):

      X_trening_fold = (treningssett_markert.iloc[train, :]).iloc[:, :-1]
      X_validering_fold = (treningssett_markert.iloc[test, :]).iloc[:, :-1]

      y_trening_fold = (treningssett_markert.iloc[train, :]).iloc[:, -1]
      y_validering_fold = (treningssett_markert.iloc[test, :]).iloc[:, -1]

      knr_pipeline.fit(X_trening_fold, y_trening_fold)

      y_validering_prediksjon = knr_pipeline.predict(X_validering_fold)

      rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
      rmse_validering_resultat.append(rmse_validering)

    rmse_validering_snitt = mean(rmse_validering_resultat)

    return rmse_validering_snitt

  if __name__ == "__main__":
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=100)

  # Bruker beste paramtere for studiet for å lage modell
  beste_parametere = study.best_params

  knr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', KNeighborsRegressor(**beste_parametere))
      ])

  # Trener modell for videre evaluering
  knr_pipeline.fit(X_trening_markert, y_trening_markert)
  y_test_prediksjon = knr_pipeline.predict(X_test)
  y_trening_prediksjon = knr_pipeline.predict(X_trening_markert)

  # Henter ut evalueringsresulateter av beste modell fra undersøkelse
  rmse_test, r2_test,\
  mae_test, mape_test,\
  rmse_trening, r2_trening,\
  mae_trening, mape_trening = evaluering_beregning(y_test,
                                                   y_test_prediksjon,
                                                   y_trening_markert,
                                                   y_trening_prediksjon)

  # Legger resulater til i datasett
  resultater = [trenings_andel,
                rmse_test, r2_test, mae_test, mape_test,
                rmse_trening, r2_trening, mae_trening, mape_trening,
                beste_parametere]

  collagen_knr_resultater_enzym.loc[len(collagen_knr_resultater_enzym)] = resultater

[I 2024-03-14 11:10:21,279] A new study created in memory with name: no-name-c7e34619-291b-4578-aa2f-65df91bc74bf
[I 2024-03-14 11:10:21,655] Trial 0 finished with value: 3.9864375847308464 and parameters: {'n_neighbors': 9, 'p': 1}. Best is trial 0 with value: 3.9864375847308464.
[I 2024-03-14 11:10:21,926] Trial 1 finished with value: 5.497047411209037 and parameters: {'n_neighbors': 1, 'p': 10}. Best is trial 0 with value: 3.9864375847308464.
[I 2024-03-14 11:10:22,213] Trial 2 finished with value: 4.265706004546676 and parameters: {'n_neighbors': 9, 'p': 5}. Best is trial 0 with value: 3.9864375847308464.
[I 2024-03-14 11:10:22,479] Trial 3 finished with value: 4.029945023880603 and parameters: {'n_neighbors': 5, 'p': 5}. Best is trial 0 with value: 3.9864375847308464.
[I 2024-03-14 11:10:22,775] Trial 4 finished with value: 4.288306811219107 and parameters: {'n_neighbors': 10, 'p': 5}. Best is trial 0 with value: 3.9864375847308464.
[I 2024-03-14 11:10:23,035] Trial 5 finished wit

In [ ]:
collagen_knr_resultater_enzym

,Trenings andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,0.3,4.529926,0.409517,3.360648,0.182378,3.198217,0.482903,2.780714,0.117323,"{'n_neighbors': 6, 'p': 1}"
1,0.4,4.592635,0.393055,3.406111,0.187850,2.814194,0.493412,2.369330,0.099013,"{'n_neighbors': 8, 'p': 1}"
2,0.5,4.066532,0.524146,2.896667,0.156093,3.311032,0.497188,2.832171,0.118927,"{'n_neighbors': 10, 'p': 1}"
3,0.6,4.040619,0.530191,3.171389,0.167215,3.700862,0.458650,2.980524,0.138399,"{'n_neighbors': 10, 'p': 1}"
4,0.7,4.364141,0.451947,3.650944,0.194573,3.535211,0.459677,2.854796,0.131780,"{'n_neighbors': 10, 'p': 1}"
5,0.8,4.004956,0.538448,3.324389,0.172808,3.742923,0.483400,3.086625,0.147483,"{'n_neighbors': 10, 'p': 1}"
6,0.9,4.411939,0.439876,3.640833,0.193614,3.596400,0.487039,2.981111,0.141053,"{'n_neighbors': 10, 'p': 1}"
7,1.0,4.198479,0.492765,3.519236,0.183033,3.527554,0.508928,2.964595,0.139694,"{'n_neighbors': 8, 'p': 10}"


### **Support Vector Regressor**

In [ ]:
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:
    ubenyttet_andel = (1.0-trenings_andel)
    treningssett_markert, enzymtyper_treningssett_markert,\
    _, _, treningssett_umarkert, _, treningssett\
    = trening_testsett_oppdeling_enzym(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  X_trening = treningssett.iloc[:, :-1]
  X_trening_markert = treningssett_markert.iloc[:, :-1]
  X_trening_umarkert = treningssett_umarkert.iloc[:, :-1]
  y_trening_markert = treningssett_markert.iloc[:, -1]

  # KNR
  def objective(trial):

    parametere = {
          'kernel': trial.suggest_categorical('kernel',["linear", "poly", "rbf"]),
          'C': trial.suggest_int('C', 1.0, 10),
          'degree': trial.suggest_int('degree', 1, 10),
          'gamma': trial.suggest_categorical('gamma', ['scale', 'auto'])

      }

    svr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', SVR(**parametere))
      ])

    splitt = 5
    gjentagelser = 10

    rskf = RepeatedStratifiedKFold(n_splits = splitt,
                                   n_repeats=gjentagelser,
                                   random_state=random_seed)

    rmse_validering_resultat = []

    for train, test in rskf.split(treningssett_markert,
                                  enzymtyper_treningssett_markert):

      X_trening_fold = (treningssett_markert.iloc[train, :]).iloc[:, :-1]
      X_validering_fold = (treningssett_markert.iloc[test, :]).iloc[:, :-1]

      y_trening_fold = (treningssett_markert.iloc[train, :]).iloc[:, -1]
      y_validering_fold = (treningssett_markert.iloc[test, :]).iloc[:, -1]

      svr_pipeline.fit(X_trening_fold, y_trening_fold)

      y_validering_prediksjon = svr_pipeline.predict(X_validering_fold)

      rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
      rmse_validering_resultat.append(rmse_validering)

    rmse_validering_snitt = mean(rmse_validering_resultat)

    return rmse_validering_snitt

  if __name__ == "__main__":
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=100)

  # Bruker beste paramtere for studiet for å lage modell
  beste_parametere = study.best_params

  svr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', SVR(**beste_parametere))
      ])

  # Trener modell for videre evaluering
  svr_pipeline.fit(X_trening_markert, y_trening_markert)
  y_test_prediksjon = knr_pipeline.predict(X_test)
  y_trening_prediksjon = knr_pipeline.predict(X_trening_markert)

  # Henter ut evalueringsresulateter av beste modell fra undersøkelse
  rmse_test, r2_test,\
  mae_test, mape_test,\
  rmse_trening, r2_trening,\
  mae_trening, mape_trening = evaluering_beregning(y_test,
                                                   y_test_prediksjon,
                                                   y_trening_markert,
                                                   y_trening_prediksjon)

  # Legger resulater til i datasett
  resultater = [trenings_andel,
                rmse_test, r2_test, mae_test, mape_test,
                rmse_trening, r2_trening, mae_trening, mape_trening,
                beste_parametere]

  collagen_svr_resultater_enzym.loc[len(collagen_svr_resultater_enzym)] = resultater

[I 2024-03-14 11:14:13,472] A new study created in memory with name: no-name-a3dd63f6-684d-4605-9974-3dcf28cbf9c7
[I 2024-03-14 11:14:13,739] Trial 0 finished with value: 12.98875740298761 and parameters: {'kernel': 'poly', 'C': 6, 'degree': 7, 'gamma': 'auto'}. Best is trial 0 with value: 12.98875740298761.
[I 2024-03-14 11:14:13,982] Trial 1 finished with value: 23.20492269016048 and parameters: {'kernel': 'poly', 'C': 10, 'degree': 8, 'gamma': 'auto'}. Best is trial 0 with value: 12.98875740298761.
[I 2024-03-14 11:14:14,334] Trial 2 finished with value: 3.8742351287868875 and parameters: {'kernel': 'rbf', 'C': 5, 'degree': 3, 'gamma': 'scale'}. Best is trial 2 with value: 3.8742351287868875.
[I 2024-03-14 11:14:14,659] Trial 3 finished with value: 4.298016557852498 and parameters: {'kernel': 'rbf', 'C': 2, 'degree': 6, 'gamma': 'scale'}. Best is trial 2 with value: 3.8742351287868875.
[I 2024-03-14 11:14:15,004] Trial 4 finished with value: 21.33711854885179 and parameters: {'kerne

In [ ]:
collagen_svr_resultater_enzym

,Trenings andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,0.3,4.198479,0.492765,3.519236,0.183033,3.710587,0.303949,3.261071,0.133597,"{'kernel': 'rbf', 'C': 8, 'degree': 4, 'gamma'..."
1,0.4,4.198479,0.492765,3.519236,0.183033,3.132483,0.372340,2.636071,0.107420,"{'kernel': 'rbf', 'C': 5, 'degree': 7, 'gamma'..."
2,0.5,4.198479,0.492765,3.519236,0.183033,3.595595,0.407047,3.054036,0.128065,"{'kernel': 'rbf', 'C': 10, 'degree': 7, 'gamma..."
3,0.6,4.198479,0.492765,3.519236,0.183033,3.594365,0.489358,2.940744,0.133869,"{'kernel': 'linear', 'C': 1, 'degree': 10, 'ga..."
4,0.7,4.198479,0.492765,3.519236,0.183033,3.504037,0.469165,2.904362,0.128713,"{'kernel': 'linear', 'C': 1, 'degree': 7, 'gam..."
5,0.8,4.198479,0.492765,3.519236,0.183033,3.532262,0.539915,2.912344,0.138864,"{'kernel': 'linear', 'C': 1, 'degree': 1, 'gam..."
6,0.9,4.198479,0.492765,3.519236,0.183033,3.451214,0.527619,2.858770,0.132173,"{'kernel': 'poly', 'C': 4, 'degree': 1, 'gamma..."
7,1.0,4.198479,0.492765,3.519236,0.183033,3.527554,0.508928,2.964595,0.139694,"{'kernel': 'rbf', 'C': 3, 'degree': 1, 'gamma'..."


## Alternativ 2: Et testsett fordelt på dag og kontinuitet

In [ ]:
test_andel = 0.2
collagen_trening_markert, collagen_test,\
 _, _ = trening_testsett_oppdeling_dag(rå_data,
                                       collagen_data,
                                       test_andel=test_andel,
                                       trening_krav=True)

X_trening_markert = collagen_trening_markert.iloc[:, :-1]
X_test = collagen_test.iloc[:, :-1]
y_trening_markert = collagen_trening_markert.iloc[:, -1]
y_test = collagen_test.iloc[:, -1]

**RandomForestRegressor**

Hyperparameteroptimalisering med Optuna

In [ ]:
def objective(trial):
  parametere = {
      'n_estimators': trial.suggest_int('n_estimators', 100, 200),
      'max_depth': trial.suggest_int('max_depth', 1, 10)
  }
  rfr_pipeline = Pipeline([
      ('skalerer', StandardScaler()),
      ('modell', RandomForestRegressor(**parametere,
                                       random_state=random_seed))
  ])

  # Manuell Kryssvalidering med Kfold og optimalisering ift. gjennomsnittlig rmse

  splitt = 5

  treningssett, valideringssett = trening_validering_oppdeling_dag(rå_data,
                                                                    collagen_trening_markert,
                                                                    n_splitt=splitt,
                                                                    trening_krav=True)
  rmse_validering_resultat = []

  for i, j in zip(treningssett, valideringssett):
    train = treningssett[i]
    val = valideringssett[j]

    X_trening_fold = (collagen_trening_markert.loc[train, :]).iloc[:, :-1]
    X_validering_fold = (collagen_trening_markert.loc[val, :]).iloc[:, :-1]

    y_trening_fold = (collagen_trening_markert.loc[train, :]).iloc[:, -1]
    y_validering_fold = (collagen_trening_markert.loc[val, :]).iloc[:, -1]

    rfr_pipeline.fit(X_trening_fold, y_trening_fold)

    y_validering_prediksjon = rfr_pipeline.predict(X_validering_fold)
    rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
    rmse_validering_resultat.append(rmse_validering)

  rmse_validering_snitt = mean(rmse_validering_resultat)

  return rmse_validering_snitt

if __name__ == "__main__":
  study = optuna.create_study(direction='minimize')
  study.optimize(objective, n_trials=100)

plot_optimization_history(study)

[I 2024-02-27 12:55:00,692] A new study created in memory with name: no-name-dead5878-7d35-45ec-afc1-c3bc6fb7eeae
[I 2024-02-27 12:55:02,972] Trial 0 finished with value: 3.200556663005775 and parameters: {'n_estimators': 172, 'max_depth': 10}. Best is trial 0 with value: 3.200556663005775.
[I 2024-02-27 12:55:04,351] Trial 1 finished with value: 3.5590960201980373 and parameters: {'n_estimators': 109, 'max_depth': 2}. Best is trial 0 with value: 3.200556663005775.
[I 2024-02-27 12:55:06,260] Trial 2 finished with value: 3.2397528638298247 and parameters: {'n_estimators': 180, 'max_depth': 7}. Best is trial 0 with value: 3.200556663005775.
[I 2024-02-27 12:55:08,127] Trial 3 finished with value: 3.5799845892864286 and parameters: {'n_estimators': 185, 'max_depth': 2}. Best is trial 0 with value: 3.200556663005775.
[I 2024-02-27 12:55:10,090] Trial 4 finished with value: 3.26589568073929 and parameters: {'n_estimators': 190, 'max_depth': 6}. Best is trial 0 with value: 3.200556663005775

In [ ]:
# Bruker beste paramtere for studiet for å lage modell
beste_parametere = study.best_params

rfr_pipeline = Pipeline([
    ('skalerer', StandardScaler()),
    ('modell', RandomForestRegressor(**beste_parametere,
                                     random_state=random_seed))
])

# Trener modell for videre evaluering
rfr_pipeline.fit(X_trening_markert, y_trening_markert)
y_test_prediksjon = rfr_pipeline.predict(X_test)
y_trening_prediksjon = rfr_pipeline.predict(X_trening_markert)

# Henter ut evalueringsresulateter av beste modell fra undersøkelse
rmse_test, r2_test, mae_test, mape_test,\
rmse_trening, r2_trening, mae_trening, mape_trening\
= evaluering_beregning(y_test,
                        y_test_prediksjon,
                        y_trening_markert,
                        y_trening_prediksjon)


# Legger resulater til i datasett
resultater = [test_andel,
              rmse_test, r2_test, mae_test, mape_test,
              rmse_trening, r2_trening, mae_trening, mape_trening,
              beste_parametere]

collagen_rfr_resultater_dag = pd.DataFrame(columns = ["Test andel",
                                                      "RMSE test",
                                                      "R2 test",
                                                      "MAE test",
                                                      "MAPE test",
                                                      "RMSE trening",
                                                      "R2 trening",
                                                      "MAE trening",
                                                      "MAPE trening",
                                                      "Beste parametere"])

collagen_rfr_resultater_dag.loc[len(collagen_rfr_resultater_dag)] = resultater

In [ ]:
collagen_rfr_resultater_dag

,Test andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,0.2,2.814854,0.664399,2.59057,0.121022,1.305851,0.939135,1.058789,0.051623,"{'n_estimators': 120, 'max_depth': 10}"


## Alternativ 2: Flere testandeler

In [ ]:
collagen_rfr_resultater_dag = pd.DataFrame(columns = ["Test andel",
                                                      "RMSE test",
                                                      "R2 test",
                                                      "MAE test",
                                                      "MAPE test",
                                                      "RMSE trening",
                                                      "R2 trening",
                                                      "MAE trening",
                                                      "MAPE trening",
                                                      "Beste parametere"])

test_andeler = [0.1, 0.2]

for test_andel in test_andeler:
  collagen_trening_markert, collagen_test, _, _ = trening_testsett_oppdeling_dag(rå_data,
                                                                         collagen_data,
                                                                         test_andel=test_andel)

  X_trening_markert = collagen_trening_markert.iloc[:, :-1]
  X_test = collagen_test.iloc[:, :-1]
  y_trening_markert = collagen_trening_markert.iloc[:, -1]
  y_test = collagen_test.iloc[:, -1]

  def objective(trial):
    # Manuell Kryssvalidering med Kfold og optimalisering ift. gjennomsnittlig rmse

    splitt = 5

    treningssett, valideringssett = trening_validering_oppdeling_dag(rå_data,
                                                                      collagen_trening_markert,
                                                                      n_splitt=splitt,
                                                                      trening_krav=True)
    rmse_validering_resultat = []

    for i, j in zip(treningssett, valideringssett):
      train = treningssett[i]
      val = valideringssett[j]

      X_trening_fold = (collagen_trening_markert.loc[train, :]).iloc[:, :-1]
      X_validering_fold = (collagen_trening_markert.loc[val, :]).iloc[:, :-1]

      y_trening_fold = (collagen_trening_markert.loc[train, :]).iloc[:, -1]
      y_validering_fold = (collagen_trening_markert.loc[val, :]).iloc[:, -1]

      rfr_pipeline.fit(X_trening_fold, y_trening_fold)

      y_validering_prediksjon = rfr_pipeline.predict(X_validering_fold)
      rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
      rmse_validering_resultat.append(rmse_validering)

    rmse_validering_snitt = mean(rmse_validering_resultat)

    return rmse_validering_snitt

  if __name__ == "__main__":
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=100)


  # Bruker beste paramtere for studiet for å lage modell
  beste_parametere = study.best_params

  rfr_pipeline = Pipeline([
      ('skalerer', StandardScaler()),
      ('modell', RandomForestRegressor(**beste_parametere, random_state=random_seed))
  ])

  # Trener modell for videre evaluering
  rfr_pipeline.fit(X_trening_markert, y_trening_markert)
  y_test_prediksjon = rfr_pipeline.predict(X_test)
  y_trening_prediksjon = rfr_pipeline.predict(X_trening_markert)

  # Henter ut evalueringsresulateter av beste modell fra undersøkelse
  rmse_test, r2_test, mae_test, mape_test,\
  rmse_trening, r2_trening, mae_trening, mape_trening\
  = evaluering_beregning(y_test,
                          y_test_prediksjon,
                          y_trening_markert,
                          y_trening_prediksjon)


  # Legger resulater til i datasett
  resultater = [test_andel,
                rmse_test, r2_test, mae_test, mape_test,
                rmse_trening, r2_trening, mae_trening, mape_trening,
                beste_parametere]

  collagen_rfr_resultater_dag.loc[len(collagen_rfr_resultater_dag)] = resultater

In [ ]:
collagen_rfr_resultater_dag

,Test andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,0.1,1.193553,0.918419,1.091318,0.051979,1.218213,0.943346,0.999373,0.047705,"{'n_estimators': 180, 'max_depth': 7}"
1,0.2,1.126552,0.946246,0.935842,0.042563,1.173786,0.947403,0.939597,0.044500,"{'n_estimators': 105, 'max_depth': 10}"


## Alternativ 2: Flere treningssandeler

In [ ]:
collagen_rfr_resultater_dag = pd.DataFrame(columns = ["Trenings andel",
                                                        "RMSE test",
                                                        "R2 test",
                                                        "MAE test",
                                                        "MAPE test",
                                                        "RMSE trening",
                                                        "R2 trening",
                                                        "MAE trening",
                                                        "MAPE trening",
                                                        "Beste parametere"])

collagen_knr_resultater_dag = pd.DataFrame(columns = ["Trenings andel",
                                                         "RMSE test",
                                                         "R2 test",
                                                         "MAE test",
                                                         "MAPE test",
                                                         "RMSE trening",
                                                         "R2 trening",
                                                         "MAE trening",
                                                         "MAPE trening",
                                                         "Beste parametere"])

collagen_svr_resultater_dag = pd.DataFrame(columns = ["Trenings andel",
                                                         "RMSE test",
                                                         "R2 test",
                                                         "MAE test",
                                                         "MAPE test",
                                                         "RMSE trening",
                                                         "R2 trening",
                                                         "MAE trening",
                                                         "MAPE trening",
                                                         "Beste parametere"])



test_andel = 0.2
trening_markert, test,\
trening_umarkert, trening = trening_testsett_oppdeling_dag(rå_data,
                                                          collagen_data,
                                                          test_andel=test_andel,
                                                          trening_krav=True)

X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]

# Treningsandeler
trenings_andeler = [0.8, 0.9, 1.0]

### **RandomForestRegressor**

In [ ]:
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  X_trening = treningssett.iloc[:, :-1]
  X_trening_markert = treningssett_markert.iloc[:, :-1]
  X_trening_umarkert = treningssett_umarkert.iloc[:, :-1]
  y_trening_markert = treningssett_markert.iloc[:, -1]

  # RandomForestRegressor
  def objective(trial):

    parametere = {
          'n_estimators': trial.suggest_int('n_estimators', 100, 200),
          'max_depth': trial.suggest_int('max_depth', 1, 10)
      }
    rfr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', RandomForestRegressor(**parametere,
                                           random_state=random_seed))
      ])

    splitt = 2

    treningssett, valideringssett = trening_validering_oppdeling_dag(rå_data,
                                                                     treningssett_markert,
                                                                     n_splitt=splitt,
                                                                     trening_krav=True)
    rmse_validering_resultat = []

    for i, j in zip(treningssett, valideringssett):
      train = treningssett[i]
      val = valideringssett[j]

      X_trening_fold = (trening_markert.loc[train, :]).iloc[:, :-1]
      X_validering_fold = (trening_markert.loc[val, :]).iloc[:, :-1]

      y_trening_fold = (trening_markert.loc[train, :]).iloc[:, -1]
      y_validering_fold = (trening_markert.loc[val, :]).iloc[:, -1]

      rfr_pipeline.fit(X_trening_fold, y_trening_fold)

      y_validering_prediksjon = rfr_pipeline.predict(X_validering_fold)
      rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
      rmse_validering_resultat.append(rmse_validering)

    rmse_validering_snitt = mean(rmse_validering_resultat)

    return rmse_validering_snitt


  if __name__ == "__main__":
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=100)


  # Bruker beste paramtere for studiet for å lage modell
  beste_parametere = study.best_params

  rfr_pipeline = Pipeline([
      ('skalerer', StandardScaler()),
      ('modell', RandomForestRegressor(**beste_parametere,
                                       random_state=random_seed))
  ])

  # Trener modell for videre evaluering
  rfr_pipeline.fit(X_trening_markert, y_trening_markert)
  y_test_prediksjon = rfr_pipeline.predict(X_test)
  y_trening_prediksjon = rfr_pipeline.predict(X_trening_markert)

  # Henter ut evalueringsresulateter av beste modell fra undersøkelse
  rmse_test, r2_test, mae_test, mape_test,\
  rmse_trening, r2_trening, mae_trening, mape_trening\
  = evaluering_beregning(y_test,
                         y_test_prediksjon,
                         y_trening_markert,
                         y_trening_prediksjon)


  # Legger resulater til i datasett
  resultater = [trenings_andel,
                rmse_test, r2_test, mae_test, mape_test,
                rmse_trening, r2_trening, mae_trening, mape_trening,
                beste_parametere]

  collagen_rfr_resultater_dag.loc[len(collagen_rfr_resultater_dag)] = resultater

[I 2024-03-13 23:00:12,138] A new study created in memory with name: no-name-a01ed145-38a3-4d09-b37a-676683fa9876
[I 2024-03-13 23:00:13,646] Trial 0 finished with value: 3.796984304282677 and parameters: {'n_estimators': 184, 'max_depth': 6}. Best is trial 0 with value: 3.796984304282677.
[I 2024-03-13 23:00:14,991] Trial 1 finished with value: 3.753802900291549 and parameters: {'n_estimators': 168, 'max_depth': 7}. Best is trial 1 with value: 3.753802900291549.
[I 2024-03-13 23:00:16,184] Trial 2 finished with value: 3.774555885235837 and parameters: {'n_estimators': 149, 'max_depth': 6}. Best is trial 1 with value: 3.753802900291549.
[I 2024-03-13 23:00:17,481] Trial 3 finished with value: 4.268282413840528 and parameters: {'n_estimators': 148, 'max_depth': 2}. Best is trial 1 with value: 3.753802900291549.
[I 2024-03-13 23:00:18,545] Trial 4 finished with value: 5.023118309586517 and parameters: {'n_estimators': 131, 'max_depth': 1}. Best is trial 1 with value: 3.753802900291549.
[

KeyboardInterrupt: 

In [ ]:
colagen_rfr_resultater_dag

### **KNeighborRegressor**

In [ ]:
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  X_trening = treningssett.iloc[:, :-1]
  X_trening_markert = treningssett_markert.iloc[:, :-1]
  X_trening_umarkert = treningssett_umarkert.iloc[:, :-1]
  y_trening_markert = treningssett_markert.iloc[:, -1]

  # KneigborsRegressor
  def objective(trial):

    parametere = {
          'n_neighbors': trial.suggest_int('n_neighbors', 1, 20),
          'p': trial.suggest_int('p', 1, 20)
      }
    knr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', KNeighborsRegressor(**parametere))
      ])


    splitt = 2

    treningssett, valideringssett = trening_validering_oppdeling_dag(rå_data,
                                                                      treningssett_markert,
                                                                      n_splitt=splitt,
                                                                      trening_krav=True)
    rmse_validering_resultat = []

    for i, j in zip(treningssett, valideringssett):
      train = treningssett[i]
      val = valideringssett[j]

      X_trening_fold = (trening_markert.loc[train, :]).iloc[:, :-1]
      X_validering_fold = (trening_markert.loc[val, :]).iloc[:, :-1]

      y_trening_fold = (trening_markert.loc[train, :]).iloc[:, -1]
      y_validering_fold = (trening_markert.loc[val, :]).iloc[:, -1]

      knr_pipeline.fit(X_trening_fold, y_trening_fold)

      y_validering_prediksjon = knr_pipeline.predict(X_validering_fold)
      rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
      rmse_validering_resultat.append(rmse_validering)

    rmse_validering_snitt = mean(rmse_validering_resultat)

    return rmse_validering_snitt


  if __name__ == "__main__":
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=100)


  # Bruker beste paramtere for studiet for å lage modell
  beste_parametere = study.best_params

  knr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', KNeighborsRegressor(**beste_parametere))
      ])

  # Trener modell for videre evaluering
  knr_pipeline.fit(X_trening_markert, y_trening_markert)
  y_test_prediksjon = knr_pipeline.predict(X_test)
  y_trening_prediksjon = knr_pipeline.predict(X_trening_markert)

  # Henter ut evalueringsresulateter av beste modell fra undersøkelse
  rmse_test, r2_test,\
  mae_test, mape_test,\
  rmse_trening, r2_trening,\
  mae_trening, mape_trening = evaluering_beregning(y_test,
                                                   y_test_prediksjon,
                                                   y_trening_markert,
                                                   y_trening_prediksjon)

  # Legger resulater til i datasett
  resultater = [trenings_andel,
                rmse_test, r2_test, mae_test, mape_test,
                rmse_trening, r2_trening, mae_trening, mape_trening,
                beste_parametere]

  collagen_knr_resultater_dag.loc[len(collagen_knr_resultater_dag)] = resultater

[I 2024-03-13 23:00:35,673] A new study created in memory with name: no-name-5da69154-224d-4077-ae0b-05c4196416ec
[I 2024-03-13 23:00:36,500] Trial 0 finished with value: 5.004266801871374 and parameters: {'n_neighbors': 14, 'p': 15}. Best is trial 0 with value: 5.004266801871374.
[I 2024-03-13 23:00:38,278] Trial 1 finished with value: 5.157710462165955 and parameters: {'n_neighbors': 19, 'p': 5}. Best is trial 0 with value: 5.004266801871374.
[I 2024-03-13 23:00:39,495] Trial 2 finished with value: 4.5030614348004 and parameters: {'n_neighbors': 11, 'p': 1}. Best is trial 2 with value: 4.5030614348004.
[I 2024-03-13 23:00:40,943] Trial 3 finished with value: 4.688114562376997 and parameters: {'n_neighbors': 11, 'p': 13}. Best is trial 2 with value: 4.5030614348004.
[I 2024-03-13 23:00:41,890] Trial 4 finished with value: 4.5859244572412505 and parameters: {'n_neighbors': 8, 'p': 17}. Best is trial 2 with value: 4.5030614348004.
[I 2024-03-13 23:00:42,998] Trial 5 finished with value:

KeyboardInterrupt: 

In [ ]:
collagen_knr_resultater_dag

### **Support Vector Regressor**

In [ ]:
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  X_trening = treningssett.iloc[:, :-1]
  X_trening_markert = treningssett_markert.iloc[:, :-1]
  X_trening_umarkert = treningssett_umarkert.iloc[:, :-1]
  y_trening_markert = treningssett_markert.iloc[:, -1]

  # SupportVectorRegressor
  def objective(trial):

    parametere = {
          'kernel': trial.suggest_categorical('kernel',["linear", "poly", "rbf"]),
          'C': trial.suggest_categorical('C', [0.1, 1, 10]),
          'degree': trial.suggest_int('degree', 1, 10)
      }

    svr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', SVR(**parametere))
      ])


    splitt = 2

    treningssett, valideringssett = trening_validering_oppdeling_dag(rå_data,
                                                                      treningssett_markert,
                                                                      n_splitt=splitt,
                                                                      trening_krav=True)
    rmse_validering_resultat = []

    for i, j in zip(treningssett, valideringssett):
      train = treningssett[i]
      val = valideringssett[j]

      X_trening_fold = (trening_markert.loc[train, :]).iloc[:, :-1]
      X_validering_fold = (trening_markert.loc[val, :]).iloc[:, :-1]

      y_trening_fold = (trening_markert.loc[train, :]).iloc[:, -1]
      y_validering_fold = (trening_markert.loc[val, :]).iloc[:, -1]

      svr_pipeline.fit(X_trening_fold, y_trening_fold)

      y_validering_prediksjon = svr_pipeline.predict(X_validering_fold)
      rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
      rmse_validering_resultat.append(rmse_validering)

    rmse_validering_snitt = mean(rmse_validering_resultat)

    return rmse_validering_snitt


  if __name__ == "__main__":
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=100)


  # Bruker beste paramtere for studiet for å lage modell
  beste_parametere = study.best_params

  svr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', SVR(**beste_parametere))
      ])

  # Trener modell for videre evaluering
  svr_pipeline.fit(X_trening_markert, y_trening_markert)
  y_test_prediksjon = svr_pipeline.predict(X_test)
  y_trening_prediksjon = svr_pipeline.predict(X_trening_markert)

  # Henter ut evalueringsresulateter av beste modell fra undersøkelse
  rmse_test, r2_test,\
  mae_test, mape_test,\
  rmse_trening, r2_trening,\
  mae_trening, mape_trening = evaluering_beregning(y_test,
                                                   y_test_prediksjon,
                                                   y_trening_markert,
                                                   y_trening_prediksjon)

  # Legger resulater til i datasett
  resultater = [trenings_andel,
                rmse_test, r2_test, mae_test, mape_test,
                rmse_trening, r2_trening, mae_trening, mape_trening,
                beste_parametere]

  collagen_svr_resultater_dag.loc[len(collagen_svr_resultater_dag)] = resultater

[I 2024-03-13 23:01:11,665] A new study created in memory with name: no-name-e979a9f0-8f1c-422b-affd-ab69514c3002
[I 2024-03-13 23:01:12,270] Trial 0 finished with value: 4.546318014245733 and parameters: {'kernel': 'linear', 'C': 0.1, 'degree': 10}. Best is trial 0 with value: 4.546318014245733.
[I 2024-03-13 23:01:12,818] Trial 1 finished with value: 9.297602010201523 and parameters: {'kernel': 'linear', 'C': 10, 'degree': 9}. Best is trial 0 with value: 4.546318014245733.
[I 2024-03-13 23:01:13,385] Trial 2 finished with value: 9.297602010201523 and parameters: {'kernel': 'linear', 'C': 10, 'degree': 3}. Best is trial 0 with value: 4.546318014245733.
[I 2024-03-13 23:01:14,013] Trial 3 finished with value: 9.818960714969776 and parameters: {'kernel': 'linear', 'C': 1, 'degree': 9}. Best is trial 0 with value: 4.546318014245733.
[I 2024-03-13 23:01:14,738] Trial 4 finished with value: 5.774523272878266 and parameters: {'kernel': 'rbf', 'C': 0.1, 'degree': 8}. Best is trial 0 with val

KeyboardInterrupt: 

In [ ]:
collagen_svr_resultater_dag

## Designproduksjon

## Alternativ 1: Flere treningsandeler

In [ ]:
collagen_rfr_resultater_enzym = pd.DataFrame(columns = ["Trenings andel",
                                                        "RMSE test",
                                                        "R2 test",
                                                        "MAE test",
                                                        "MAPE test",
                                                        "RMSE trening",
                                                        "R2 trening",
                                                        "MAE trening",
                                                        "MAPE trening",
                                                        "Beste parametere"])

collagen_knr_resultater_enzym = pd.DataFrame(columns = ["Trenings andel",
                                                         "RMSE test",
                                                         "R2 test",
                                                         "MAE test",
                                                         "MAPE test",
                                                         "RMSE trening",
                                                         "R2 trening",
                                                         "MAE trening",
                                                         "MAPE trening",
                                                         "Beste parametere"])

collagen_svr_resultater_enzym = pd.DataFrame(columns = ["Trenings andel",
                                                         "RMSE test",
                                                         "R2 test",
                                                         "MAE test",
                                                         "MAPE test",
                                                         "RMSE trening",
                                                         "R2 trening",
                                                         "MAE trening",
                                                         "MAPE trening",
                                                         "Beste parametere"])



# Setter fast testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert,test, _, trening_umarkert,\
_, trening = trening_testsett_oppdeling_enzym(rå_data,
                                              collagen_data_design,
                                              test_andel=test_andel)

# Treningssett
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]


# Treningsandeler
treningsandeler = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

### **RandomForestRegressor**

In [ ]:
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  X_trening = treningssett.iloc[:, :-1]
  X_trening_markert = treningssett_markert.iloc[:, :-1]
  X_trening_umarkert = treningssett_umarkert.iloc[:, :-1]
  y_trening_markert = treningssett_markert.iloc[:, -1]

  # RandomForestRegressor
  def objective(trial):

    parametere = {
          'n_estimators': trial.suggest_int('n_estimators', 100, 200),
          'max_depth': trial.suggest_int('max_depth', 1, 10)
      }
    rfr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', RandomForestRegressor(**parametere,
                                           random_state=random_seed))
      ])

    splitt = 5
    gjentagelser = 10

    rskf = RepeatedStratifiedKFold(n_splits = splitt,
                                  n_repeats=gjentagelser,
                                  random_state=random_seed)

    rmse_validering_resultat = []

    for train, test in rskf.split(treningssett_markert,
                                  enzymtyper_treningssett_markert):

      X_trening_fold = (treningssett_markert.iloc[train, :]).iloc[:, :-1]
      X_validering_fold = (treningssett_markert.iloc[test, :]).iloc[:, :-1]

      y_trening_fold = (treningssett_markert.iloc[train, :]).iloc[:, -1]
      y_validering_fold = (treningssett_markert.iloc[test, :]).iloc[:, -1]

      rfr_pipeline.fit(X_trening_fold, y_trening_fold)

      y_validering_prediksjon = rfr_pipeline.predict(X_validering_fold)
      rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
      rmse_validering_resultat.append(rmse_validering)

    rmse_validering_snitt = mean(rmse_validering_resultat)

    return rmse_validering_snitt

  if __name__ == "__main__":
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=10)


  # Bruker beste paramtere for studiet for å lage modell
  beste_parametere = study.best_params

  rfr_pipeline = Pipeline([
      ('skalerer', StandardScaler()),
      ('modell', RandomForestRegressor(**beste_parametere,
                                       random_state=random_seed))
  ])

  # Trener modell for videre evaluering
  rfr_pipeline.fit(X_trening_markert, y_trening_markert)
  y_test_prediksjon = rfr_pipeline.predict(X_test)
  y_trening_prediksjon = rfr_pipeline.predict(X_trening_markert)

  # Henter ut evalueringsresulateter av beste modell fra undersøkelse
  rmse_test, r2_test, mae_test, mape_test,\
  rmse_trening, r2_trening, mae_trening, mape_trening\
  = evaluering_beregning(y_test,
                         y_test_prediksjon,
                         y_trening_markert,
                         y_trening_prediksjon)


  # Legger resulater til i datasett
  resultater = [trenings_andel,
                rmse_test, r2_test, mae_test, mape_test,
                rmse_trening, r2_trening, mae_trening, mape_trening,
                beste_parametere]

  collagen_rfr_resultater_enzym.loc[len(collagen_rfr_resultater_enzym)] = resultater

[I 2024-03-13 17:45:11,277] A new study created in memory with name: no-name-d2796126-3636-44a8-8395-4f06d6eda11b
[I 2024-03-13 17:45:24,466] Trial 0 finished with value: 3.7923223907848262 and parameters: {'n_estimators': 194, 'max_depth': 2}. Best is trial 0 with value: 3.7923223907848262.
[I 2024-03-13 17:45:35,363] Trial 1 finished with value: 3.8187620559221918 and parameters: {'n_estimators': 153, 'max_depth': 5}. Best is trial 0 with value: 3.7923223907848262.
[I 2024-03-13 17:45:42,158] Trial 2 finished with value: 3.7937260175047838 and parameters: {'n_estimators': 102, 'max_depth': 1}. Best is trial 0 with value: 3.7923223907848262.
[I 2024-03-13 17:45:52,401] Trial 3 finished with value: 3.824007038643008 and parameters: {'n_estimators': 148, 'max_depth': 5}. Best is trial 0 with value: 3.7923223907848262.
[I 2024-03-13 17:46:04,871] Trial 4 finished with value: 3.799048568907229 and parameters: {'n_estimators': 179, 'max_depth': 8}. Best is trial 0 with value: 3.79232239078

In [ ]:
collagen_rfr_resultater_enzym

,Trenings andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,0.2,4.900275,0.309019,4.090229,0.216316,1.712875,0.826292,1.380287,0.058717,"{'n_estimators': 194, 'max_depth': 2}"
1,0.3,4.499416,0.417444,3.777991,0.198470,1.107255,0.938020,0.801087,0.034505,"{'n_estimators': 188, 'max_depth': 10}"
2,0.4,4.530213,0.409442,3.626124,0.195302,0.941608,0.943286,0.774610,0.032186,"{'n_estimators': 186, 'max_depth': 8}"
3,0.5,4.567976,0.399556,3.636077,0.193478,1.058123,0.948649,0.826429,0.035172,"{'n_estimators': 161, 'max_depth': 7}"
4,0.6,4.301157,0.467652,3.515187,0.185020,1.445862,0.917372,1.084352,0.052138,"{'n_estimators': 187, 'max_depth': 4}"
5,0.7,4.079096,0.521201,3.417341,0.179191,1.260303,0.931329,0.965931,0.046862,"{'n_estimators': 173, 'max_depth': 7}"
6,0.8,3.658807,0.614784,3.078608,0.157573,1.261491,0.941319,0.983510,0.048987,"{'n_estimators': 147, 'max_depth': 6}"
7,0.9,3.753000,0.594695,3.167425,0.161638,1.149998,0.947550,0.929524,0.044950,"{'n_estimators': 102, 'max_depth': 7}"
8,1.0,4.062459,0.525099,3.561316,0.182568,1.804048,0.871562,1.457385,0.070768,"{'n_estimators': 190, 'max_depth': 4}"


### **KNeighborsRegressor**

In [ ]:
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:
    ubenyttet_andel = (1.0-trenings_andel)
    treningssett_markert, enzymtyper_treningssett_markert,\
    _, _, treningssett_umarkert, _, treningssett\
    = trening_testsett_oppdeling_enzym(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  X_trening = treningssett.iloc[:, :-1]
  X_trening_markert = treningssett_markert.iloc[:, :-1]
  X_trening_umarkert = treningssett_umarkert.iloc[:, :-1]
  y_trening_markert = treningssett_markert.iloc[:, -1]

  # KNR
  def objective(trial):

    parametere = {
          'n_neighbors': trial.suggest_int('n_neighbors', 1, 10),
          'p': trial.suggest_int('p', 1, 10)
      }
    knr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', KNeighborsRegressor(**parametere))
      ])

    splitt = 5
    gjentagelser = 10

    rskf = RepeatedStratifiedKFold(n_splits = splitt,
                                   n_repeats=gjentagelser,
                                   random_state=random_seed)

    rmse_validering_resultat = []

    for train, test in rskf.split(treningssett_markert,
                                  enzymtyper_treningssett_markert):

      X_trening_fold = (treningssett_markert.iloc[train, :]).iloc[:, :-1]
      X_validering_fold = (treningssett_markert.iloc[test, :]).iloc[:, :-1]

      y_trening_fold = (treningssett_markert.iloc[train, :]).iloc[:, -1]
      y_validering_fold = (treningssett_markert.iloc[test, :]).iloc[:, -1]

      knr_pipeline.fit(X_trening_fold, y_trening_fold)

      y_validering_prediksjon = knr_pipeline.predict(X_validering_fold)

      rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
      rmse_validering_resultat.append(rmse_validering)

    rmse_validering_snitt = mean(rmse_validering_resultat)

    return rmse_validering_snitt

  if __name__ == "__main__":
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=10)

  # Bruker beste paramtere for studiet for å lage modell
  beste_parametere = study.best_params

  knr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', KNeighborsRegressor(**beste_parametere))
      ])

  # Trener modell for videre evaluering
  knr_pipeline.fit(X_trening_markert, y_trening_markert)
  y_test_prediksjon = knr_pipeline.predict(X_test)
  y_trening_prediksjon = knr_pipeline.predict(X_trening_markert)

  # Henter ut evalueringsresulateter av beste modell fra undersøkelse
  rmse_test, r2_test,\
  mae_test, mape_test,\
  rmse_trening, r2_trening,\
  mae_trening, mape_trening = evaluering_beregning(y_test,
                                                   y_test_prediksjon,
                                                   y_trening_markert,
                                                   y_trening_prediksjon)

  # Legger resulater til i datasett
  resultater = [trenings_andel,
                rmse_test, r2_test, mae_test, mape_test,
                rmse_trening, r2_trening, mae_trening, mape_trening,
                beste_parametere]

  collagen_knr_resultater_enzym.loc[len(collagen_knr_resultater_enzym)] = resultater

[I 2024-03-13 18:16:12,459] A new study created in memory with name: no-name-e6a846de-a735-4749-a433-8d09fad5e96b
[I 2024-03-13 18:16:12,849] Trial 0 finished with value: 5.15348184336347 and parameters: {'n_neighbors': 1, 'p': 8}. Best is trial 0 with value: 5.15348184336347.
[I 2024-03-13 18:16:13,237] Trial 1 finished with value: 4.957840751833551 and parameters: {'n_neighbors': 1, 'p': 2}. Best is trial 1 with value: 4.957840751833551.
[I 2024-03-13 18:16:13,604] Trial 2 finished with value: 5.115380215250371 and parameters: {'n_neighbors': 1, 'p': 6}. Best is trial 1 with value: 4.957840751833551.
[I 2024-03-13 18:16:13,992] Trial 3 finished with value: 4.254923914653543 and parameters: {'n_neighbors': 2, 'p': 2}. Best is trial 3 with value: 4.254923914653543.
[I 2024-03-13 18:16:14,361] Trial 4 finished with value: 4.254923914653543 and parameters: {'n_neighbors': 2, 'p': 2}. Best is trial 3 with value: 4.254923914653543.
[I 2024-03-13 18:16:14,786] Trial 5 finished with value: 3

In [ ]:
collagen_knr_resultater_enzym

,Trenings andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,0.2,4.756410,0.348996,3.633472,0.198921,3.094609,0.433005,2.619821,0.113225,"{'n_neighbors': 4, 'p': 8}"
1,0.3,4.750257,0.350679,3.543796,0.193311,3.561828,0.358640,3.181508,0.134417,"{'n_neighbors': 6, 'p': 3}"
2,0.4,4.372122,0.449940,3.134938,0.177458,2.973665,0.434372,2.549921,0.104547,"{'n_neighbors': 9, 'p': 3}"
3,0.5,3.956061,0.549649,2.982222,0.161934,3.356377,0.483322,2.802036,0.118554,"{'n_neighbors': 8, 'p': 2}"
4,0.6,4.575462,0.397586,3.731019,0.202458,3.528880,0.507795,2.773929,0.126463,"{'n_neighbors': 6, 'p': 9}"
5,0.7,4.454108,0.429117,3.617569,0.194244,3.646405,0.425153,2.931505,0.132224,"{'n_neighbors': 8, 'p': 4}"
6,0.8,4.432152,0.434732,3.523944,0.189401,3.869625,0.447834,3.230625,0.150631,"{'n_neighbors': 10, 'p': 7}"
7,0.9,4.411939,0.439876,3.640833,0.193614,3.596400,0.487039,2.981111,0.141053,"{'n_neighbors': 10, 'p': 1}"
8,1.0,4.307204,0.466154,3.551728,0.187885,3.625567,0.481260,3.055603,0.143697,"{'n_neighbors': 9, 'p': 9}"


### **Support Vector Regressor**

In [ ]:
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:
    ubenyttet_andel = (1.0-trenings_andel)
    treningssett_markert, enzymtyper_treningssett_markert,\
    _, _, treningssett_umarkert, _, treningssett\
    = trening_testsett_oppdeling_enzym(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  X_trening = treningssett.iloc[:, :-1]
  X_trening_markert = treningssett_markert.iloc[:, :-1]
  X_trening_umarkert = treningssett_umarkert.iloc[:, :-1]
  y_trening_markert = treningssett_markert.iloc[:, -1]

  # KNR
  def objective(trial):

    parametere = {
          'kernel': trial.suggest_categorical('kernel',["linear", "poly", "rbf"]),
          'C': trial.suggest_int('C', 1.0, 10),
          'degree': trial.suggest_int('degree', 1, 10),
          'gamma': trial.suggest_categorical('gamma', ['scale', 'auto'])

      }

    svr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', SVR(**parametere))
      ])

    splitt = 5
    gjentagelser = 10

    rskf = RepeatedStratifiedKFold(n_splits = splitt,
                                   n_repeats=gjentagelser,
                                   random_state=random_seed)

    rmse_validering_resultat = []

    for train, test in rskf.split(treningssett_markert,
                                  enzymtyper_treningssett_markert):

      X_trening_fold = (treningssett_markert.iloc[train, :]).iloc[:, :-1]
      X_validering_fold = (treningssett_markert.iloc[test, :]).iloc[:, :-1]

      y_trening_fold = (treningssett_markert.iloc[train, :]).iloc[:, -1]
      y_validering_fold = (treningssett_markert.iloc[test, :]).iloc[:, -1]

      svr_pipeline.fit(X_trening_fold, y_trening_fold)

      y_validering_prediksjon = svr_pipeline.predict(X_validering_fold)

      rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
      rmse_validering_resultat.append(rmse_validering)

    rmse_validering_snitt = mean(rmse_validering_resultat)

    return rmse_validering_snitt

  if __name__ == "__main__":
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=10)

  # Bruker beste paramtere for studiet for å lage modell
  beste_parametere = study.best_params

  svr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', SVR(**beste_parametere))
      ])

  # Trener modell for videre evaluering
  svr_pipeline.fit(X_trening_markert, y_trening_markert)
  y_test_prediksjon = knr_pipeline.predict(X_test)
  y_trening_prediksjon = knr_pipeline.predict(X_trening_markert)

  # Henter ut evalueringsresulateter av beste modell fra undersøkelse
  rmse_test, r2_test,\
  mae_test, mape_test,\
  rmse_trening, r2_trening,\
  mae_trening, mape_trening = evaluering_beregning(y_test,
                                                   y_test_prediksjon,
                                                   y_trening_markert,
                                                   y_trening_prediksjon)

  # Legger resulater til i datasett
  resultater = [trenings_andel,
                rmse_test, r2_test, mae_test, mape_test,
                rmse_trening, r2_trening, mae_trening, mape_trening,
                beste_parametere]

  collagen_svr_resultater_enzym.loc[len(collagen_svr_resultater_enzym)] = resultater

[I 2024-03-13 18:17:34,978] A new study created in memory with name: no-name-53f85498-fc78-490c-9fc5-304b74636e31
[I 2024-03-13 18:17:35,619] Trial 0 finished with value: 3.7904467345636945 and parameters: {'kernel': 'rbf', 'C': 5, 'degree': 1, 'gamma': 'auto'}. Best is trial 0 with value: 3.7904467345636945.
[I 2024-03-13 18:17:36,239] Trial 1 finished with value: 3.62551513007233 and parameters: {'kernel': 'rbf', 'C': 8, 'degree': 3, 'gamma': 'auto'}. Best is trial 1 with value: 3.62551513007233.
[I 2024-03-13 18:17:36,861] Trial 2 finished with value: 3.7904467345636945 and parameters: {'kernel': 'rbf', 'C': 5, 'degree': 7, 'gamma': 'auto'}. Best is trial 1 with value: 3.62551513007233.
[I 2024-03-13 18:17:37,644] Trial 3 finished with value: 4.432757379200256 and parameters: {'kernel': 'linear', 'C': 5, 'degree': 1, 'gamma': 'auto'}. Best is trial 1 with value: 3.62551513007233.
[I 2024-03-13 18:17:38,831] Trial 4 finished with value: 3.670453529024988 and parameters: {'kernel': 'r

In [ ]:
collagen_svr_resultater_enzym

,Trenings andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,0.2,4.307204,0.466154,3.551728,0.187885,3.385852,0.321260,2.945079,0.123292,"{'kernel': 'rbf', 'C': 8, 'degree': 3, 'gamma'..."
1,0.3,4.307204,0.466154,3.551728,0.187885,3.842720,0.253494,3.376349,0.138902,"{'kernel': 'rbf', 'C': 7, 'degree': 3, 'gamma'..."
2,0.4,4.307204,0.466154,3.551728,0.187885,3.185608,0.350869,2.676111,0.109010,"{'kernel': 'rbf', 'C': 5, 'degree': 10, 'gamma..."
3,0.5,4.307204,0.466154,3.551728,0.187885,3.713764,0.367432,3.167238,0.133206,"{'kernel': 'poly', 'C': 4, 'degree': 1, 'gamma..."
4,0.6,4.307204,0.466154,3.551728,0.187885,3.700362,0.458797,3.038148,0.138249,"{'kernel': 'linear', 'C': 1, 'degree': 1, 'gam..."
5,0.7,4.307204,0.466154,3.551728,0.187885,3.601753,0.439145,2.996644,0.132832,"{'kernel': 'poly', 'C': 10, 'degree': 1, 'gamm..."
6,0.8,4.307204,0.466154,3.551728,0.187885,3.663763,0.505021,3.036032,0.144439,"{'kernel': 'poly', 'C': 5, 'degree': 1, 'gamma..."
7,0.9,4.307204,0.466154,3.551728,0.187885,3.558889,0.497684,2.958536,0.136675,"{'kernel': 'rbf', 'C': 3, 'degree': 5, 'gamma'..."
8,1.0,4.307204,0.466154,3.551728,0.187885,3.625567,0.481260,3.055603,0.143697,"{'kernel': 'rbf', 'C': 10, 'degree': 3, 'gamma..."


## Alternativ 2: Flere treningssandeler

In [ ]:
collagen_rfr_resultater_dag = pd.DataFrame(columns = ["Trenings andel",
                                                        "RMSE test",
                                                        "R2 test",
                                                        "MAE test",
                                                        "MAPE test",
                                                        "RMSE trening",
                                                        "R2 trening",
                                                        "MAE trening",
                                                        "MAPE trening",
                                                        "Beste parametere"])

collagen_knr_resultater_dag = pd.DataFrame(columns = ["Trenings andel",
                                                         "RMSE test",
                                                         "R2 test",
                                                         "MAE test",
                                                         "MAPE test",
                                                         "RMSE trening",
                                                         "R2 trening",
                                                         "MAE trening",
                                                         "MAPE trening",
                                                         "Beste parametere"])

collagen_svr_resultater_dag = pd.DataFrame(columns = ["Trenings andel",
                                                         "RMSE test",
                                                         "R2 test",
                                                         "MAE test",
                                                         "MAPE test",
                                                         "RMSE trening",
                                                         "R2 trening",
                                                         "MAE trening",
                                                         "MAPE trening",
                                                         "Beste parametere"])

collagen_coreg_resultater_dag = pd.DataFrame(columns = ["Trenings andel",
                                                         "RMSE test",
                                                         "R2 test",
                                                         "MAE test",
                                                         "MAPE test",
                                                         "RMSE trening",
                                                         "R2 trening",
                                                         "MAE trening",
                                                         "MAPE trening",
                                                         "Beste parametere"])

collagen_bdh_resultater_dag = pd.DataFrame(columns = ["Trenings andel",
                                                         "RMSE test",
                                                         "R2 test",
                                                         "MAE test",
                                                         "MAPE test",
                                                         "RMSE trening",
                                                         "R2 trening",
                                                         "MAE trening",
                                                         "MAPE trening",
                                                         "Beste parametere"])


test_andel = 0.2
trening_markert, test,\
trening_umarkert, trening = trening_testsett_oppdeling_dag(rå_data,
                                                          collagen_data_design,
                                                          test_andel=test_andel,
                                                          trening_krav=True)

X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]

# Treningsandeler
trenings_andeler = [0.8, 0.9, 1.0]

### **RandomForestRegressor**

In [ ]:
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  X_trening = treningssett.iloc[:, :-1]
  X_trening_markert = treningssett_markert.iloc[:, :-1]
  X_trening_umarkert = treningssett_umarkert.iloc[:, :-1]
  y_trening_markert = treningssett_markert.iloc[:, -1]

  # RandomForestRegressor
  def objective(trial):

    parametere = {
          'n_estimators': trial.suggest_int('n_estimators', 100, 200),
          'max_depth': trial.suggest_int('max_depth', 1, 10)
      }
    rfr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', RandomForestRegressor(**parametere,
                                           random_state=random_seed))
      ])

    splitt = 2

    treningssett, valideringssett = trening_validering_oppdeling_dag(rå_data,
                                                                     treningssett_markert,
                                                                     n_splitt=splitt,
                                                                     trening_krav=True)
    rmse_validering_resultat = []

    for i, j in zip(treningssett, valideringssett):
      train = treningssett[i]
      val = valideringssett[j]

      X_trening_fold = (trening_markert.loc[train, :]).iloc[:, :-1]
      X_validering_fold = (trening_markert.loc[val, :]).iloc[:, :-1]

      y_trening_fold = (trening_markert.loc[train, :]).iloc[:, -1]
      y_validering_fold = (trening_markert.loc[val, :]).iloc[:, -1]

      rfr_pipeline.fit(X_trening_fold, y_trening_fold)

      y_validering_prediksjon = rfr_pipeline.predict(X_validering_fold)
      rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
      rmse_validering_resultat.append(rmse_validering)

    rmse_validering_snitt = mean(rmse_validering_resultat)

    return rmse_validering_snitt


  if __name__ == "__main__":
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=100)


  # Bruker beste paramtere for studiet for å lage modell
  beste_parametere = study.best_params

  rfr_pipeline = Pipeline([
      ('skalerer', StandardScaler()),
      ('modell', RandomForestRegressor(**beste_parametere,
                                       random_state=random_seed))
  ])

  # Trener modell for videre evaluering
  rfr_pipeline.fit(X_trening_markert, y_trening_markert)
  y_test_prediksjon = rfr_pipeline.predict(X_test)
  y_trening_prediksjon = rfr_pipeline.predict(X_trening_markert)

  # Henter ut evalueringsresulateter av beste modell fra undersøkelse
  rmse_test, r2_test, mae_test, mape_test,\
  rmse_trening, r2_trening, mae_trening, mape_trening\
  = evaluering_beregning(y_test,
                         y_test_prediksjon,
                         y_trening_markert,
                         y_trening_prediksjon)


  # Legger resulater til i datasett
  resultater = [trenings_andel,
                rmse_test, r2_test, mae_test, mape_test,
                rmse_trening, r2_trening, mae_trening, mape_trening,
                beste_parametere]

  collagen_rfr_resultater_dag.loc[len(collagen_rfr_resultater_dag)] = resultater

[I 2024-03-13 23:00:12,138] A new study created in memory with name: no-name-a01ed145-38a3-4d09-b37a-676683fa9876
[I 2024-03-13 23:00:13,646] Trial 0 finished with value: 3.796984304282677 and parameters: {'n_estimators': 184, 'max_depth': 6}. Best is trial 0 with value: 3.796984304282677.
[I 2024-03-13 23:00:14,991] Trial 1 finished with value: 3.753802900291549 and parameters: {'n_estimators': 168, 'max_depth': 7}. Best is trial 1 with value: 3.753802900291549.
[I 2024-03-13 23:00:16,184] Trial 2 finished with value: 3.774555885235837 and parameters: {'n_estimators': 149, 'max_depth': 6}. Best is trial 1 with value: 3.753802900291549.
[I 2024-03-13 23:00:17,481] Trial 3 finished with value: 4.268282413840528 and parameters: {'n_estimators': 148, 'max_depth': 2}. Best is trial 1 with value: 3.753802900291549.
[I 2024-03-13 23:00:18,545] Trial 4 finished with value: 5.023118309586517 and parameters: {'n_estimators': 131, 'max_depth': 1}. Best is trial 1 with value: 3.753802900291549.
[

KeyboardInterrupt: 

In [ ]:
collagen_rfr_resultater_dag

### **KNeighborRegressor**

In [ ]:
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  X_trening = treningssett.iloc[:, :-1]
  X_trening_markert = treningssett_markert.iloc[:, :-1]
  X_trening_umarkert = treningssett_umarkert.iloc[:, :-1]
  y_trening_markert = treningssett_markert.iloc[:, -1]

  # KneigborsRegressor
  def objective(trial):

    parametere = {
          'n_neighbors': trial.suggest_int('n_neighbors', 1, 20),
          'p': trial.suggest_int('p', 1, 20)
      }
    knr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', KNeighborsRegressor(**parametere))
      ])


    splitt = 2

    treningssett, valideringssett = trening_validering_oppdeling_dag(rå_data,
                                                                      treningssett_markert,
                                                                      n_splitt=splitt,
                                                                      trening_krav=True)
    rmse_validering_resultat = []

    for i, j in zip(treningssett, valideringssett):
      train = treningssett[i]
      val = valideringssett[j]

      X_trening_fold = (trening_markert.loc[train, :]).iloc[:, :-1]
      X_validering_fold = (trening_markert.loc[val, :]).iloc[:, :-1]

      y_trening_fold = (trening_markert.loc[train, :]).iloc[:, -1]
      y_validering_fold = (trening_markert.loc[val, :]).iloc[:, -1]

      knr_pipeline.fit(X_trening_fold, y_trening_fold)

      y_validering_prediksjon = knr_pipeline.predict(X_validering_fold)
      rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
      rmse_validering_resultat.append(rmse_validering)

    rmse_validering_snitt = mean(rmse_validering_resultat)

    return rmse_validering_snitt


  if __name__ == "__main__":
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=100)


  # Bruker beste paramtere for studiet for å lage modell
  beste_parametere = study.best_params

  knr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', KNeighborsRegressor(**beste_parametere))
      ])

  # Trener modell for videre evaluering
  knr_pipeline.fit(X_trening_markert, y_trening_markert)
  y_test_prediksjon = knr_pipeline.predict(X_test)
  y_trening_prediksjon = knr_pipeline.predict(X_trening_markert)

  # Henter ut evalueringsresulateter av beste modell fra undersøkelse
  rmse_test, r2_test,\
  mae_test, mape_test,\
  rmse_trening, r2_trening,\
  mae_trening, mape_trening = evaluering_beregning(y_test,
                                                   y_test_prediksjon,
                                                   y_trening_markert,
                                                   y_trening_prediksjon)

  # Legger resulater til i datasett
  resultater = [trenings_andel,
                rmse_test, r2_test, mae_test, mape_test,
                rmse_trening, r2_trening, mae_trening, mape_trening,
                beste_parametere]

  collagen_knr_resultater_dag.loc[len(collagen_knr_resultater_dag)] = resultater

[I 2024-03-13 23:00:35,673] A new study created in memory with name: no-name-5da69154-224d-4077-ae0b-05c4196416ec
[I 2024-03-13 23:00:36,500] Trial 0 finished with value: 5.004266801871374 and parameters: {'n_neighbors': 14, 'p': 15}. Best is trial 0 with value: 5.004266801871374.
[I 2024-03-13 23:00:38,278] Trial 1 finished with value: 5.157710462165955 and parameters: {'n_neighbors': 19, 'p': 5}. Best is trial 0 with value: 5.004266801871374.
[I 2024-03-13 23:00:39,495] Trial 2 finished with value: 4.5030614348004 and parameters: {'n_neighbors': 11, 'p': 1}. Best is trial 2 with value: 4.5030614348004.
[I 2024-03-13 23:00:40,943] Trial 3 finished with value: 4.688114562376997 and parameters: {'n_neighbors': 11, 'p': 13}. Best is trial 2 with value: 4.5030614348004.
[I 2024-03-13 23:00:41,890] Trial 4 finished with value: 4.5859244572412505 and parameters: {'n_neighbors': 8, 'p': 17}. Best is trial 2 with value: 4.5030614348004.
[I 2024-03-13 23:00:42,998] Trial 5 finished with value:

KeyboardInterrupt: 

In [ ]:
collagen_knr_resultater_dag

### **Support Vector Regressor**

In [ ]:
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  X_trening = treningssett.iloc[:, :-1]
  X_trening_markert = treningssett_markert.iloc[:, :-1]
  X_trening_umarkert = treningssett_umarkert.iloc[:, :-1]
  y_trening_markert = treningssett_markert.iloc[:, -1]

  # SupportVectorRegressor
  def objective(trial):

    parametere = {
          'kernel': trial.suggest_categorical('kernel',["linear", "poly", "rbf"]),
          'C': trial.suggest_categorical('C', [0.1, 1, 10]),
          'degree': trial.suggest_int('degree', 1, 10)
      }

    svr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', SVR(**parametere))
      ])


    splitt = 2

    treningssett, valideringssett = trening_validering_oppdeling_dag(rå_data,
                                                                      treningssett_markert,
                                                                      n_splitt=splitt,
                                                                      trening_krav=True)
    rmse_validering_resultat = []

    for i, j in zip(treningssett, valideringssett):
      train = treningssett[i]
      val = valideringssett[j]

      X_trening_fold = (trening_markert.loc[train, :]).iloc[:, :-1]
      X_validering_fold = (trening_markert.loc[val, :]).iloc[:, :-1]

      y_trening_fold = (trening_markert.loc[train, :]).iloc[:, -1]
      y_validering_fold = (trening_markert.loc[val, :]).iloc[:, -1]

      svr_pipeline.fit(X_trening_fold, y_trening_fold)

      y_validering_prediksjon = svr_pipeline.predict(X_validering_fold)
      rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
      rmse_validering_resultat.append(rmse_validering)

    rmse_validering_snitt = mean(rmse_validering_resultat)

    return rmse_validering_snitt


  if __name__ == "__main__":
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=100)


  # Bruker beste paramtere for studiet for å lage modell
  beste_parametere = study.best_params

  svr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', SVR(**beste_parametere))
      ])

  # Trener modell for videre evaluering
  svr_pipeline.fit(X_trening_markert, y_trening_markert)
  y_test_prediksjon = svr_pipeline.predict(X_test)
  y_trening_prediksjon = svr_pipeline.predict(X_trening_markert)

  # Henter ut evalueringsresulateter av beste modell fra undersøkelse
  rmse_test, r2_test,\
  mae_test, mape_test,\
  rmse_trening, r2_trening,\
  mae_trening, mape_trening = evaluering_beregning(y_test,
                                                   y_test_prediksjon,
                                                   y_trening_markert,
                                                   y_trening_prediksjon)

  # Legger resulater til i datasett
  resultater = [trenings_andel,
                rmse_test, r2_test, mae_test, mape_test,
                rmse_trening, r2_trening, mae_trening, mape_trening,
                beste_parametere]

  collagen_svr_resultater_dag.loc[len(collagen_svr_resultater_dag)] = resultater

[I 2024-03-13 23:01:11,665] A new study created in memory with name: no-name-e979a9f0-8f1c-422b-affd-ab69514c3002
[I 2024-03-13 23:01:12,270] Trial 0 finished with value: 4.546318014245733 and parameters: {'kernel': 'linear', 'C': 0.1, 'degree': 10}. Best is trial 0 with value: 4.546318014245733.
[I 2024-03-13 23:01:12,818] Trial 1 finished with value: 9.297602010201523 and parameters: {'kernel': 'linear', 'C': 10, 'degree': 9}. Best is trial 0 with value: 4.546318014245733.
[I 2024-03-13 23:01:13,385] Trial 2 finished with value: 9.297602010201523 and parameters: {'kernel': 'linear', 'C': 10, 'degree': 3}. Best is trial 0 with value: 4.546318014245733.
[I 2024-03-13 23:01:14,013] Trial 3 finished with value: 9.818960714969776 and parameters: {'kernel': 'linear', 'C': 1, 'degree': 9}. Best is trial 0 with value: 4.546318014245733.
[I 2024-03-13 23:01:14,738] Trial 4 finished with value: 5.774523272878266 and parameters: {'kernel': 'rbf', 'C': 0.1, 'degree': 8}. Best is trial 0 with val

KeyboardInterrupt: 

In [ ]:
collagen_svr_resultater_dag

# NIR målinger ekskludert

In [ ]:
collagen_data_uten_NIR = collagen_data.drop(columns=['NIRfat', 'NIRwater', 'NIRash'])
collagen_data_design_uten_NIR = collagen_data_design.drop(columns=['NIRfat', 'NIRwater', 'NIRash'])

## Hele datasett

## Alternativ 1: Flere treningsandeler

In [ ]:
collagen_rfr_resultater_enzym = pd.DataFrame(columns = ["Trenings andel",
                                                        "RMSE test",
                                                        "R2 test",
                                                        "MAE test",
                                                        "MAPE test",
                                                        "RMSE trening",
                                                        "R2 trening",
                                                        "MAE trening",
                                                        "MAPE trening",
                                                        "Beste parametere"])

collagen_knr_resultater_enzym = pd.DataFrame(columns = ["Trenings andel",
                                                         "RMSE test",
                                                         "R2 test",
                                                         "MAE test",
                                                         "MAPE test",
                                                         "RMSE trening",
                                                         "R2 trening",
                                                         "MAE trening",
                                                         "MAPE trening",
                                                         "Beste parametere"])

collagen_svr_resultater_enzym = pd.DataFrame(columns = ["Trenings andel",
                                                         "RMSE test",
                                                         "R2 test",
                                                         "MAE test",
                                                         "MAPE test",
                                                         "RMSE trening",
                                                         "R2 trening",
                                                         "MAE trening",
                                                         "MAPE trening",
                                                         "Beste parametere"])



# Setter fast testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert,test, _, trening_umarkert,\
_, trening = trening_testsett_oppdeling_enzym(rå_data,
                                              collagen_data_uten_NIR,
                                              test_andel=test_andel)

# Treningssett
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]


# Treningsandeler
treningsandeler = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

### **RandomForestRegressor**

In [ ]:
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  X_trening = treningssett.iloc[:, :-1]
  X_trening_markert = treningssett_markert.iloc[:, :-1]
  X_trening_umarkert = treningssett_umarkert.iloc[:, :-1]
  y_trening_markert = treningssett_markert.iloc[:, -1]

  # RandomForestRegressor
  def objective(trial):

    parametere = {
          'n_estimators': trial.suggest_int('n_estimators', 100, 200),
          'max_depth': trial.suggest_int('max_depth', 1, 10)
      }
    rfr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', RandomForestRegressor(**parametere,
                                           random_state=random_seed))
      ])

    splitt = 5
    gjentagelser = 10

    rskf = RepeatedStratifiedKFold(n_splits = splitt,
                                  n_repeats=gjentagelser,
                                  random_state=random_seed)

    rmse_validering_resultat = []

    for train, test in rskf.split(treningssett_markert,
                                  enzymtyper_treningssett_markert):

      X_trening_fold = (treningssett_markert.iloc[train, :]).iloc[:, :-1]
      X_validering_fold = (treningssett_markert.iloc[test, :]).iloc[:, :-1]

      y_trening_fold = (treningssett_markert.iloc[train, :]).iloc[:, -1]
      y_validering_fold = (treningssett_markert.iloc[test, :]).iloc[:, -1]

      rfr_pipeline.fit(X_trening_fold, y_trening_fold)

      y_validering_prediksjon = rfr_pipeline.predict(X_validering_fold)
      rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
      rmse_validering_resultat.append(rmse_validering)

    rmse_validering_snitt = mean(rmse_validering_resultat)

    return rmse_validering_snitt

  if __name__ == "__main__":
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=10)


  # Bruker beste paramtere for studiet for å lage modell
  beste_parametere = study.best_params

  rfr_pipeline = Pipeline([
      ('skalerer', StandardScaler()),
      ('modell', RandomForestRegressor(**beste_parametere,
                                       random_state=random_seed))
  ])

  # Trener modell for videre evaluering
  rfr_pipeline.fit(X_trening_markert, y_trening_markert)
  y_test_prediksjon = rfr_pipeline.predict(X_test)
  y_trening_prediksjon = rfr_pipeline.predict(X_trening_markert)

  # Henter ut evalueringsresulateter av beste modell fra undersøkelse
  rmse_test, r2_test, mae_test, mape_test,\
  rmse_trening, r2_trening, mae_trening, mape_trening\
  = evaluering_beregning(y_test,
                         y_test_prediksjon,
                         y_trening_markert,
                         y_trening_prediksjon)


  # Legger resulater til i datasett
  resultater = [trenings_andel,
                rmse_test, r2_test, mae_test, mape_test,
                rmse_trening, r2_trening, mae_trening, mape_trening,
                beste_parametere]

  collagen_rfr_resultater_enzym.loc[len(collagen_rfr_resultater_enzym)] = resultater

[I 2024-03-13 23:56:48,370] A new study created in memory with name: no-name-7b5157e8-d4f4-4d86-8d3b-447c2b5cfef2
[I 2024-03-13 23:57:01,578] Trial 0 finished with value: 4.5633961398406155 and parameters: {'n_estimators': 163, 'max_depth': 6}. Best is trial 0 with value: 4.5633961398406155.
[I 2024-03-13 23:57:12,519] Trial 1 finished with value: 4.551427492261231 and parameters: {'n_estimators': 152, 'max_depth': 10}. Best is trial 1 with value: 4.551427492261231.
[I 2024-03-13 23:57:22,691] Trial 2 finished with value: 4.508431590257096 and parameters: {'n_estimators': 141, 'max_depth': 1}. Best is trial 2 with value: 4.508431590257096.
[I 2024-03-13 23:57:32,332] Trial 3 finished with value: 4.541297365275147 and parameters: {'n_estimators': 141, 'max_depth': 5}. Best is trial 2 with value: 4.508431590257096.
[I 2024-03-13 23:57:43,059] Trial 4 finished with value: 4.559436522010408 and parameters: {'n_estimators': 161, 'max_depth': 7}. Best is trial 2 with value: 4.508431590257096

KeyboardInterrupt: 

In [ ]:
collagen_rfr_resultater_enzym

### **KNeighborsRegressor**

In [ ]:
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:
    ubenyttet_andel = (1.0-trenings_andel)
    treningssett_markert, enzymtyper_treningssett_markert,\
    _, _, treningssett_umarkert, _, treningssett\
    = trening_testsett_oppdeling_enzym(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  X_trening = treningssett.iloc[:, :-1]
  X_trening_markert = treningssett_markert.iloc[:, :-1]
  X_trening_umarkert = treningssett_umarkert.iloc[:, :-1]
  y_trening_markert = treningssett_markert.iloc[:, -1]

  # KNR
  def objective(trial):

    parametere = {
          'n_neighbors': trial.suggest_int('n_neighbors', 1, 10),
          'p': trial.suggest_int('p', 1, 10)
      }
    knr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', KNeighborsRegressor(**parametere))
      ])

    splitt = 5
    gjentagelser = 10

    rskf = RepeatedStratifiedKFold(n_splits = splitt,
                                   n_repeats=gjentagelser,
                                   random_state=random_seed)

    rmse_validering_resultat = []

    for train, test in rskf.split(treningssett_markert,
                                  enzymtyper_treningssett_markert):

      X_trening_fold = (treningssett_markert.iloc[train, :]).iloc[:, :-1]
      X_validering_fold = (treningssett_markert.iloc[test, :]).iloc[:, :-1]

      y_trening_fold = (treningssett_markert.iloc[train, :]).iloc[:, -1]
      y_validering_fold = (treningssett_markert.iloc[test, :]).iloc[:, -1]

      knr_pipeline.fit(X_trening_fold, y_trening_fold)

      y_validering_prediksjon = knr_pipeline.predict(X_validering_fold)

      rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
      rmse_validering_resultat.append(rmse_validering)

    rmse_validering_snitt = mean(rmse_validering_resultat)

    return rmse_validering_snitt

  if __name__ == "__main__":
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=10)

  # Bruker beste paramtere for studiet for å lage modell
  beste_parametere = study.best_params

  knr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', KNeighborsRegressor(**beste_parametere))
      ])

  # Trener modell for videre evaluering
  knr_pipeline.fit(X_trening_markert, y_trening_markert)
  y_test_prediksjon = knr_pipeline.predict(X_test)
  y_trening_prediksjon = knr_pipeline.predict(X_trening_markert)

  # Henter ut evalueringsresulateter av beste modell fra undersøkelse
  rmse_test, r2_test,\
  mae_test, mape_test,\
  rmse_trening, r2_trening,\
  mae_trening, mape_trening = evaluering_beregning(y_test,
                                                   y_test_prediksjon,
                                                   y_trening_markert,
                                                   y_trening_prediksjon)

  # Legger resulater til i datasett
  resultater = [trenings_andel,
                rmse_test, r2_test, mae_test, mape_test,
                rmse_trening, r2_trening, mae_trening, mape_trening,
                beste_parametere]

  collagen_knr_resultater_enzym.loc[len(collagen_knr_resultater_enzym)] = resultater

[I 2024-03-13 23:29:14,910] A new study created in memory with name: no-name-30fb351a-2bea-4df7-8406-e5725959b831
[I 2024-03-13 23:29:15,279] Trial 0 finished with value: 5.640850067557779 and parameters: {'n_neighbors': 2, 'p': 4}. Best is trial 0 with value: 5.640850067557779.
[I 2024-03-13 23:29:15,690] Trial 1 finished with value: 6.779294851644629 and parameters: {'n_neighbors': 1, 'p': 1}. Best is trial 0 with value: 5.640850067557779.
[I 2024-03-13 23:29:16,094] Trial 2 finished with value: 7.0054717082854205 and parameters: {'n_neighbors': 1, 'p': 6}. Best is trial 0 with value: 5.640850067557779.
[I 2024-03-13 23:29:16,490] Trial 3 finished with value: 5.162854314615598 and parameters: {'n_neighbors': 3, 'p': 2}. Best is trial 3 with value: 5.162854314615598.
[I 2024-03-13 23:29:16,951] Trial 4 finished with value: 4.623507390304366 and parameters: {'n_neighbors': 9, 'p': 5}. Best is trial 4 with value: 4.623507390304366.
[I 2024-03-13 23:29:17,420] Trial 5 finished with value

KeyboardInterrupt: 

In [ ]:
collagen_knr_resultater_enzym

### **Support Vector Regressor**

In [ ]:
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:
    ubenyttet_andel = (1.0-trenings_andel)
    treningssett_markert, enzymtyper_treningssett_markert,\
    _, _, treningssett_umarkert, _, treningssett\
    = trening_testsett_oppdeling_enzym(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  X_trening = treningssett.iloc[:, :-1]
  X_trening_markert = treningssett_markert.iloc[:, :-1]
  X_trening_umarkert = treningssett_umarkert.iloc[:, :-1]
  y_trening_markert = treningssett_markert.iloc[:, -1]

  # KNR
  def objective(trial):

    parametere = {
          'kernel': trial.suggest_categorical('kernel',["linear", "poly", "rbf"]),
          'C': trial.suggest_int('C', 1.0, 10),
          'degree': trial.suggest_int('degree', 1, 10),
          'gamma': trial.suggest_categorical('gamma', ['scale', 'auto'])

      }

    svr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', SVR(**parametere))
      ])

    splitt = 5
    gjentagelser = 10

    rskf = RepeatedStratifiedKFold(n_splits = splitt,
                                   n_repeats=gjentagelser,
                                   random_state=random_seed)

    rmse_validering_resultat = []

    for train, test in rskf.split(treningssett_markert,
                                  enzymtyper_treningssett_markert):

      X_trening_fold = (treningssett_markert.iloc[train, :]).iloc[:, :-1]
      X_validering_fold = (treningssett_markert.iloc[test, :]).iloc[:, :-1]

      y_trening_fold = (treningssett_markert.iloc[train, :]).iloc[:, -1]
      y_validering_fold = (treningssett_markert.iloc[test, :]).iloc[:, -1]

      svr_pipeline.fit(X_trening_fold, y_trening_fold)

      y_validering_prediksjon = svr_pipeline.predict(X_validering_fold)

      rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
      rmse_validering_resultat.append(rmse_validering)

    rmse_validering_snitt = mean(rmse_validering_resultat)

    return rmse_validering_snitt

  if __name__ == "__main__":
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=10)

  # Bruker beste paramtere for studiet for å lage modell
  beste_parametere = study.best_params

  svr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', SVR(**beste_parametere))
      ])

  # Trener modell for videre evaluering
  svr_pipeline.fit(X_trening_markert, y_trening_markert)
  y_test_prediksjon = knr_pipeline.predict(X_test)
  y_trening_prediksjon = knr_pipeline.predict(X_trening_markert)

  # Henter ut evalueringsresulateter av beste modell fra undersøkelse
  rmse_test, r2_test,\
  mae_test, mape_test,\
  rmse_trening, r2_trening,\
  mae_trening, mape_trening = evaluering_beregning(y_test,
                                                   y_test_prediksjon,
                                                   y_trening_markert,
                                                   y_trening_prediksjon)

  # Legger resulater til i datasett
  resultater = [trenings_andel,
                rmse_test, r2_test, mae_test, mape_test,
                rmse_trening, r2_trening, mae_trening, mape_trening,
                beste_parametere]

  collagen_svr_resultater_enzym.loc[len(collagen_svr_resultater_enzym)] = resultater

[I 2024-03-13 23:29:30,648] A new study created in memory with name: no-name-33e03e11-0b39-4388-907f-cbf1637a9544
[I 2024-03-13 23:29:31,019] Trial 0 finished with value: 4.826849926799751 and parameters: {'kernel': 'rbf', 'C': 2, 'degree': 6, 'gamma': 'auto'}. Best is trial 0 with value: 4.826849926799751.
[I 2024-03-13 23:29:31,397] Trial 1 finished with value: 5.664937065628462 and parameters: {'kernel': 'poly', 'C': 8, 'degree': 2, 'gamma': 'auto'}. Best is trial 0 with value: 4.826849926799751.
[I 2024-03-13 23:29:31,789] Trial 2 finished with value: 4.826849926799751 and parameters: {'kernel': 'rbf', 'C': 2, 'degree': 6, 'gamma': 'auto'}. Best is trial 0 with value: 4.826849926799751.
[I 2024-03-13 23:29:32,208] Trial 3 finished with value: 6.0941044619967695 and parameters: {'kernel': 'poly', 'C': 8, 'degree': 2, 'gamma': 'scale'}. Best is trial 0 with value: 4.826849926799751.
[I 2024-03-13 23:29:32,593] Trial 4 finished with value: 5.078352906209942 and parameters: {'kernel': 

KeyboardInterrupt: 

In [ ]:
collagen_svr_resultater_enzym

## Alternativ 2: Flere treningssandeler

In [ ]:
collagen_rfr_resultater_dag = pd.DataFrame(columns = ["Trenings andel",
                                                        "RMSE test",
                                                        "R2 test",
                                                        "MAE test",
                                                        "MAPE test",
                                                        "RMSE trening",
                                                        "R2 trening",
                                                        "MAE trening",
                                                        "MAPE trening",
                                                        "Beste parametere"])

collagen_knr_resultater_dag = pd.DataFrame(columns = ["Trenings andel",
                                                         "RMSE test",
                                                         "R2 test",
                                                         "MAE test",
                                                         "MAPE test",
                                                         "RMSE trening",
                                                         "R2 trening",
                                                         "MAE trening",
                                                         "MAPE trening",
                                                         "Beste parametere"])

collagen_svr_resultater_dag = pd.DataFrame(columns = ["Trenings andel",
                                                         "RMSE test",
                                                         "R2 test",
                                                         "MAE test",
                                                         "MAPE test",
                                                         "RMSE trening",
                                                         "R2 trening",
                                                         "MAE trening",
                                                         "MAPE trening",
                                                         "Beste parametere"])



test_andel = 0.2
trening_markert, test,\
trening_umarkert, trening = trening_testsett_oppdeling_dag(rå_data,
                                                          collagen_data_uten_NIR,
                                                          test_andel=test_andel,
                                                          trening_krav=True)

X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]

# Treningsandeler
trenings_andeler = [0.8, 0.9, 1.0]

### **RandomForestRegressor**

In [ ]:
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  X_trening = treningssett.iloc[:, :-1]
  X_trening_markert = treningssett_markert.iloc[:, :-1]
  X_trening_umarkert = treningssett_umarkert.iloc[:, :-1]
  y_trening_markert = treningssett_markert.iloc[:, -1]

  # RandomForestRegressor
  def objective(trial):

    parametere = {
          'n_estimators': trial.suggest_int('n_estimators', 100, 200),
          'max_depth': trial.suggest_int('max_depth', 1, 10)
      }
    rfr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', RandomForestRegressor(**parametere,
                                           random_state=random_seed))
      ])

    splitt = 2

    treningssett, valideringssett = trening_validering_oppdeling_dag(rå_data,
                                                                     treningssett_markert,
                                                                     n_splitt=splitt,
                                                                     trening_krav=True)
    rmse_validering_resultat = []

    for i, j in zip(treningssett, valideringssett):
      train = treningssett[i]
      val = valideringssett[j]

      X_trening_fold = (trening_markert.loc[train, :]).iloc[:, :-1]
      X_validering_fold = (trening_markert.loc[val, :]).iloc[:, :-1]

      y_trening_fold = (trening_markert.loc[train, :]).iloc[:, -1]
      y_validering_fold = (trening_markert.loc[val, :]).iloc[:, -1]

      rfr_pipeline.fit(X_trening_fold, y_trening_fold)

      y_validering_prediksjon = rfr_pipeline.predict(X_validering_fold)
      rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
      rmse_validering_resultat.append(rmse_validering)

    rmse_validering_snitt = mean(rmse_validering_resultat)

    return rmse_validering_snitt


  if __name__ == "__main__":
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=100)


  # Bruker beste paramtere for studiet for å lage modell
  beste_parametere = study.best_params

  rfr_pipeline = Pipeline([
      ('skalerer', StandardScaler()),
      ('modell', RandomForestRegressor(**beste_parametere,
                                       random_state=random_seed))
  ])

  # Trener modell for videre evaluering
  rfr_pipeline.fit(X_trening_markert, y_trening_markert)
  y_test_prediksjon = rfr_pipeline.predict(X_test)
  y_trening_prediksjon = rfr_pipeline.predict(X_trening_markert)

  # Henter ut evalueringsresulateter av beste modell fra undersøkelse
  rmse_test, r2_test, mae_test, mape_test,\
  rmse_trening, r2_trening, mae_trening, mape_trening\
  = evaluering_beregning(y_test,
                         y_test_prediksjon,
                         y_trening_markert,
                         y_trening_prediksjon)


  # Legger resulater til i datasett
  resultater = [trenings_andel,
                rmse_test, r2_test, mae_test, mape_test,
                rmse_trening, r2_trening, mae_trening, mape_trening,
                beste_parametere]

  collagen_rfr_resultater_dag.loc[len(collagen_rfr_resultater_dag)] = resultater

[I 2024-03-13 23:00:12,138] A new study created in memory with name: no-name-a01ed145-38a3-4d09-b37a-676683fa9876
[I 2024-03-13 23:00:13,646] Trial 0 finished with value: 3.796984304282677 and parameters: {'n_estimators': 184, 'max_depth': 6}. Best is trial 0 with value: 3.796984304282677.
[I 2024-03-13 23:00:14,991] Trial 1 finished with value: 3.753802900291549 and parameters: {'n_estimators': 168, 'max_depth': 7}. Best is trial 1 with value: 3.753802900291549.
[I 2024-03-13 23:00:16,184] Trial 2 finished with value: 3.774555885235837 and parameters: {'n_estimators': 149, 'max_depth': 6}. Best is trial 1 with value: 3.753802900291549.
[I 2024-03-13 23:00:17,481] Trial 3 finished with value: 4.268282413840528 and parameters: {'n_estimators': 148, 'max_depth': 2}. Best is trial 1 with value: 3.753802900291549.
[I 2024-03-13 23:00:18,545] Trial 4 finished with value: 5.023118309586517 and parameters: {'n_estimators': 131, 'max_depth': 1}. Best is trial 1 with value: 3.753802900291549.
[

KeyboardInterrupt: 

In [ ]:
colagen_rfr_resultater_dag

### **KNeighborRegressor**

In [ ]:
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  X_trening = treningssett.iloc[:, :-1]
  X_trening_markert = treningssett_markert.iloc[:, :-1]
  X_trening_umarkert = treningssett_umarkert.iloc[:, :-1]
  y_trening_markert = treningssett_markert.iloc[:, -1]

  # KneigborsRegressor
  def objective(trial):

    parametere = {
          'n_neighbors': trial.suggest_int('n_neighbors', 1, 20),
          'p': trial.suggest_int('p', 1, 20)
      }
    knr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', KNeighborsRegressor(**parametere))
      ])


    splitt = 2

    treningssett, valideringssett = trening_validering_oppdeling_dag(rå_data,
                                                                      treningssett_markert,
                                                                      n_splitt=splitt,
                                                                      trening_krav=True)
    rmse_validering_resultat = []

    for i, j in zip(treningssett, valideringssett):
      train = treningssett[i]
      val = valideringssett[j]

      X_trening_fold = (trening_markert.loc[train, :]).iloc[:, :-1]
      X_validering_fold = (trening_markert.loc[val, :]).iloc[:, :-1]

      y_trening_fold = (trening_markert.loc[train, :]).iloc[:, -1]
      y_validering_fold = (trening_markert.loc[val, :]).iloc[:, -1]

      knr_pipeline.fit(X_trening_fold, y_trening_fold)

      y_validering_prediksjon = knr_pipeline.predict(X_validering_fold)
      rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
      rmse_validering_resultat.append(rmse_validering)

    rmse_validering_snitt = mean(rmse_validering_resultat)

    return rmse_validering_snitt


  if __name__ == "__main__":
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=100)


  # Bruker beste paramtere for studiet for å lage modell
  beste_parametere = study.best_params

  knr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', KNeighborsRegressor(**beste_parametere))
      ])

  # Trener modell for videre evaluering
  knr_pipeline.fit(X_trening_markert, y_trening_markert)
  y_test_prediksjon = knr_pipeline.predict(X_test)
  y_trening_prediksjon = knr_pipeline.predict(X_trening_markert)

  # Henter ut evalueringsresulateter av beste modell fra undersøkelse
  rmse_test, r2_test,\
  mae_test, mape_test,\
  rmse_trening, r2_trening,\
  mae_trening, mape_trening = evaluering_beregning(y_test,
                                                   y_test_prediksjon,
                                                   y_trening_markert,
                                                   y_trening_prediksjon)

  # Legger resulater til i datasett
  resultater = [trenings_andel,
                rmse_test, r2_test, mae_test, mape_test,
                rmse_trening, r2_trening, mae_trening, mape_trening,
                beste_parametere]

  collagen_knr_resultater_dag.loc[len(collagen_knr_resultater_dag)] = resultater

[I 2024-03-13 23:00:35,673] A new study created in memory with name: no-name-5da69154-224d-4077-ae0b-05c4196416ec
[I 2024-03-13 23:00:36,500] Trial 0 finished with value: 5.004266801871374 and parameters: {'n_neighbors': 14, 'p': 15}. Best is trial 0 with value: 5.004266801871374.
[I 2024-03-13 23:00:38,278] Trial 1 finished with value: 5.157710462165955 and parameters: {'n_neighbors': 19, 'p': 5}. Best is trial 0 with value: 5.004266801871374.
[I 2024-03-13 23:00:39,495] Trial 2 finished with value: 4.5030614348004 and parameters: {'n_neighbors': 11, 'p': 1}. Best is trial 2 with value: 4.5030614348004.
[I 2024-03-13 23:00:40,943] Trial 3 finished with value: 4.688114562376997 and parameters: {'n_neighbors': 11, 'p': 13}. Best is trial 2 with value: 4.5030614348004.
[I 2024-03-13 23:00:41,890] Trial 4 finished with value: 4.5859244572412505 and parameters: {'n_neighbors': 8, 'p': 17}. Best is trial 2 with value: 4.5030614348004.
[I 2024-03-13 23:00:42,998] Trial 5 finished with value:

KeyboardInterrupt: 

In [ ]:
collagen_knr_resultater_dag

### **Support Vector Regressor**

In [ ]:
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  X_trening = treningssett.iloc[:, :-1]
  X_trening_markert = treningssett_markert.iloc[:, :-1]
  X_trening_umarkert = treningssett_umarkert.iloc[:, :-1]
  y_trening_markert = treningssett_markert.iloc[:, -1]

  # SupportVectorRegressor
  def objective(trial):

    parametere = {
          'kernel': trial.suggest_categorical('kernel',["linear", "poly", "rbf"]),
          'C': trial.suggest_categorical('C', [0.1, 1, 10]),
          'degree': trial.suggest_int('degree', 1, 10)
      }

    svr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', SVR(**parametere))
      ])


    splitt = 2

    treningssett, valideringssett = trening_validering_oppdeling_dag(rå_data,
                                                                      treningssett_markert,
                                                                      n_splitt=splitt,
                                                                      trening_krav=True)
    rmse_validering_resultat = []

    for i, j in zip(treningssett, valideringssett):
      train = treningssett[i]
      val = valideringssett[j]

      X_trening_fold = (trening_markert.loc[train, :]).iloc[:, :-1]
      X_validering_fold = (trening_markert.loc[val, :]).iloc[:, :-1]

      y_trening_fold = (trening_markert.loc[train, :]).iloc[:, -1]
      y_validering_fold = (trening_markert.loc[val, :]).iloc[:, -1]

      svr_pipeline.fit(X_trening_fold, y_trening_fold)

      y_validering_prediksjon = svr_pipeline.predict(X_validering_fold)
      rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
      rmse_validering_resultat.append(rmse_validering)

    rmse_validering_snitt = mean(rmse_validering_resultat)

    return rmse_validering_snitt


  if __name__ == "__main__":
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=100)


  # Bruker beste paramtere for studiet for å lage modell
  beste_parametere = study.best_params

  svr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', SVR(**beste_parametere))
      ])

  # Trener modell for videre evaluering
  svr_pipeline.fit(X_trening_markert, y_trening_markert)
  y_test_prediksjon = svr_pipeline.predict(X_test)
  y_trening_prediksjon = svr_pipeline.predict(X_trening_markert)

  # Henter ut evalueringsresulateter av beste modell fra undersøkelse
  rmse_test, r2_test,\
  mae_test, mape_test,\
  rmse_trening, r2_trening,\
  mae_trening, mape_trening = evaluering_beregning(y_test,
                                                   y_test_prediksjon,
                                                   y_trening_markert,
                                                   y_trening_prediksjon)

  # Legger resulater til i datasett
  resultater = [trenings_andel,
                rmse_test, r2_test, mae_test, mape_test,
                rmse_trening, r2_trening, mae_trening, mape_trening,
                beste_parametere]

  collagen_svr_resultater_dag.loc[len(collagen_svr_resultater_dag)] = resultater

[I 2024-03-13 23:01:11,665] A new study created in memory with name: no-name-e979a9f0-8f1c-422b-affd-ab69514c3002
[I 2024-03-13 23:01:12,270] Trial 0 finished with value: 4.546318014245733 and parameters: {'kernel': 'linear', 'C': 0.1, 'degree': 10}. Best is trial 0 with value: 4.546318014245733.
[I 2024-03-13 23:01:12,818] Trial 1 finished with value: 9.297602010201523 and parameters: {'kernel': 'linear', 'C': 10, 'degree': 9}. Best is trial 0 with value: 4.546318014245733.
[I 2024-03-13 23:01:13,385] Trial 2 finished with value: 9.297602010201523 and parameters: {'kernel': 'linear', 'C': 10, 'degree': 3}. Best is trial 0 with value: 4.546318014245733.
[I 2024-03-13 23:01:14,013] Trial 3 finished with value: 9.818960714969776 and parameters: {'kernel': 'linear', 'C': 1, 'degree': 9}. Best is trial 0 with value: 4.546318014245733.
[I 2024-03-13 23:01:14,738] Trial 4 finished with value: 5.774523272878266 and parameters: {'kernel': 'rbf', 'C': 0.1, 'degree': 8}. Best is trial 0 with val

KeyboardInterrupt: 

In [ ]:
collagen_svr_resultater_dag

## Designproduksjon

## Alternativ 1: Flere treningsandeler

In [ ]:
collagen_rfr_resultater_enzym = pd.DataFrame(columns = ["Trenings andel",
                                                        "RMSE test",
                                                        "R2 test",
                                                        "MAE test",
                                                        "MAPE test",
                                                        "RMSE trening",
                                                        "R2 trening",
                                                        "MAE trening",
                                                        "MAPE trening",
                                                        "Beste parametere"])

collagen_knr_resultater_enzym = pd.DataFrame(columns = ["Trenings andel",
                                                         "RMSE test",
                                                         "R2 test",
                                                         "MAE test",
                                                         "MAPE test",
                                                         "RMSE trening",
                                                         "R2 trening",
                                                         "MAE trening",
                                                         "MAPE trening",
                                                         "Beste parametere"])

collagen_svr_resultater_enzym = pd.DataFrame(columns = ["Trenings andel",
                                                         "RMSE test",
                                                         "R2 test",
                                                         "MAE test",
                                                         "MAPE test",
                                                         "RMSE trening",
                                                         "R2 trening",
                                                         "MAE trening",
                                                         "MAPE trening",
                                                         "Beste parametere"])



# Setter fast testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert,test, _, trening_umarkert,\
_, trening = trening_testsett_oppdeling_enzym(rå_data,
                                              collagen_data_design_uten_NIR,
                                              test_andel=test_andel)

# Treningssett
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]


# Treningsandeler
treningsandeler = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

### **RandomForestRegressor**

In [ ]:
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  X_trening = treningssett.iloc[:, :-1]
  X_trening_markert = treningssett_markert.iloc[:, :-1]
  X_trening_umarkert = treningssett_umarkert.iloc[:, :-1]
  y_trening_markert = treningssett_markert.iloc[:, -1]

  # RandomForestRegressor
  def objective(trial):

    parametere = {
          'n_estimators': trial.suggest_int('n_estimators', 100, 200),
          'max_depth': trial.suggest_int('max_depth', 1, 10)
      }
    rfr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', RandomForestRegressor(**parametere,
                                           random_state=random_seed))
      ])

    splitt = 5
    gjentagelser = 10

    rskf = RepeatedStratifiedKFold(n_splits = splitt,
                                  n_repeats=gjentagelser,
                                  random_state=random_seed)

    rmse_validering_resultat = []

    for train, test in rskf.split(treningssett_markert,
                                  enzymtyper_treningssett_markert):

      X_trening_fold = (treningssett_markert.iloc[train, :]).iloc[:, :-1]
      X_validering_fold = (treningssett_markert.iloc[test, :]).iloc[:, :-1]

      y_trening_fold = (treningssett_markert.iloc[train, :]).iloc[:, -1]
      y_validering_fold = (treningssett_markert.iloc[test, :]).iloc[:, -1]

      rfr_pipeline.fit(X_trening_fold, y_trening_fold)

      y_validering_prediksjon = rfr_pipeline.predict(X_validering_fold)
      rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
      rmse_validering_resultat.append(rmse_validering)

    rmse_validering_snitt = mean(rmse_validering_resultat)

    return rmse_validering_snitt

  if __name__ == "__main__":
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=10)


  # Bruker beste paramtere for studiet for å lage modell
  beste_parametere = study.best_params

  rfr_pipeline = Pipeline([
      ('skalerer', StandardScaler()),
      ('modell', RandomForestRegressor(**beste_parametere,
                                       random_state=random_seed))
  ])

  # Trener modell for videre evaluering
  rfr_pipeline.fit(X_trening_markert, y_trening_markert)
  y_test_prediksjon = rfr_pipeline.predict(X_test)
  y_trening_prediksjon = rfr_pipeline.predict(X_trening_markert)

  # Henter ut evalueringsresulateter av beste modell fra undersøkelse
  rmse_test, r2_test, mae_test, mape_test,\
  rmse_trening, r2_trening, mae_trening, mape_trening\
  = evaluering_beregning(y_test,
                         y_test_prediksjon,
                         y_trening_markert,
                         y_trening_prediksjon)


  # Legger resulater til i datasett
  resultater = [trenings_andel,
                rmse_test, r2_test, mae_test, mape_test,
                rmse_trening, r2_trening, mae_trening, mape_trening,
                beste_parametere]

  collagen_rfr_resultater_enzym.loc[len(collagen_rfr_resultater_enzym)] = resultater

[I 2024-03-13 17:45:11,277] A new study created in memory with name: no-name-d2796126-3636-44a8-8395-4f06d6eda11b
[I 2024-03-13 17:45:24,466] Trial 0 finished with value: 3.7923223907848262 and parameters: {'n_estimators': 194, 'max_depth': 2}. Best is trial 0 with value: 3.7923223907848262.
[I 2024-03-13 17:45:35,363] Trial 1 finished with value: 3.8187620559221918 and parameters: {'n_estimators': 153, 'max_depth': 5}. Best is trial 0 with value: 3.7923223907848262.
[I 2024-03-13 17:45:42,158] Trial 2 finished with value: 3.7937260175047838 and parameters: {'n_estimators': 102, 'max_depth': 1}. Best is trial 0 with value: 3.7923223907848262.
[I 2024-03-13 17:45:52,401] Trial 3 finished with value: 3.824007038643008 and parameters: {'n_estimators': 148, 'max_depth': 5}. Best is trial 0 with value: 3.7923223907848262.
[I 2024-03-13 17:46:04,871] Trial 4 finished with value: 3.799048568907229 and parameters: {'n_estimators': 179, 'max_depth': 8}. Best is trial 0 with value: 3.79232239078

In [ ]:
collagen_rfr_resultater_enzym

,Trenings andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,0.2,4.900275,0.309019,4.090229,0.216316,1.712875,0.826292,1.380287,0.058717,"{'n_estimators': 194, 'max_depth': 2}"
1,0.3,4.499416,0.417444,3.777991,0.198470,1.107255,0.938020,0.801087,0.034505,"{'n_estimators': 188, 'max_depth': 10}"
2,0.4,4.530213,0.409442,3.626124,0.195302,0.941608,0.943286,0.774610,0.032186,"{'n_estimators': 186, 'max_depth': 8}"
3,0.5,4.567976,0.399556,3.636077,0.193478,1.058123,0.948649,0.826429,0.035172,"{'n_estimators': 161, 'max_depth': 7}"
4,0.6,4.301157,0.467652,3.515187,0.185020,1.445862,0.917372,1.084352,0.052138,"{'n_estimators': 187, 'max_depth': 4}"
5,0.7,4.079096,0.521201,3.417341,0.179191,1.260303,0.931329,0.965931,0.046862,"{'n_estimators': 173, 'max_depth': 7}"
6,0.8,3.658807,0.614784,3.078608,0.157573,1.261491,0.941319,0.983510,0.048987,"{'n_estimators': 147, 'max_depth': 6}"
7,0.9,3.753000,0.594695,3.167425,0.161638,1.149998,0.947550,0.929524,0.044950,"{'n_estimators': 102, 'max_depth': 7}"
8,1.0,4.062459,0.525099,3.561316,0.182568,1.804048,0.871562,1.457385,0.070768,"{'n_estimators': 190, 'max_depth': 4}"


### **KNeighborsRegressor**

In [ ]:
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:
    ubenyttet_andel = (1.0-trenings_andel)
    treningssett_markert, enzymtyper_treningssett_markert,\
    _, _, treningssett_umarkert, _, treningssett\
    = trening_testsett_oppdeling_enzym(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  X_trening = treningssett.iloc[:, :-1]
  X_trening_markert = treningssett_markert.iloc[:, :-1]
  X_trening_umarkert = treningssett_umarkert.iloc[:, :-1]
  y_trening_markert = treningssett_markert.iloc[:, -1]

  # KNR
  def objective(trial):

    parametere = {
          'n_neighbors': trial.suggest_int('n_neighbors', 1, 10),
          'p': trial.suggest_int('p', 1, 10)
      }
    knr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', KNeighborsRegressor(**parametere))
      ])

    splitt = 5
    gjentagelser = 10

    rskf = RepeatedStratifiedKFold(n_splits = splitt,
                                   n_repeats=gjentagelser,
                                   random_state=random_seed)

    rmse_validering_resultat = []

    for train, test in rskf.split(treningssett_markert,
                                  enzymtyper_treningssett_markert):

      X_trening_fold = (treningssett_markert.iloc[train, :]).iloc[:, :-1]
      X_validering_fold = (treningssett_markert.iloc[test, :]).iloc[:, :-1]

      y_trening_fold = (treningssett_markert.iloc[train, :]).iloc[:, -1]
      y_validering_fold = (treningssett_markert.iloc[test, :]).iloc[:, -1]

      knr_pipeline.fit(X_trening_fold, y_trening_fold)

      y_validering_prediksjon = knr_pipeline.predict(X_validering_fold)

      rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
      rmse_validering_resultat.append(rmse_validering)

    rmse_validering_snitt = mean(rmse_validering_resultat)

    return rmse_validering_snitt

  if __name__ == "__main__":
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=10)

  # Bruker beste paramtere for studiet for å lage modell
  beste_parametere = study.best_params

  knr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', KNeighborsRegressor(**beste_parametere))
      ])

  # Trener modell for videre evaluering
  knr_pipeline.fit(X_trening_markert, y_trening_markert)
  y_test_prediksjon = knr_pipeline.predict(X_test)
  y_trening_prediksjon = knr_pipeline.predict(X_trening_markert)

  # Henter ut evalueringsresulateter av beste modell fra undersøkelse
  rmse_test, r2_test,\
  mae_test, mape_test,\
  rmse_trening, r2_trening,\
  mae_trening, mape_trening = evaluering_beregning(y_test,
                                                   y_test_prediksjon,
                                                   y_trening_markert,
                                                   y_trening_prediksjon)

  # Legger resulater til i datasett
  resultater = [trenings_andel,
                rmse_test, r2_test, mae_test, mape_test,
                rmse_trening, r2_trening, mae_trening, mape_trening,
                beste_parametere]

  collagen_knr_resultater_enzym.loc[len(collagen_knr_resultater_enzym)] = resultater

[I 2024-03-13 18:16:12,459] A new study created in memory with name: no-name-e6a846de-a735-4749-a433-8d09fad5e96b
[I 2024-03-13 18:16:12,849] Trial 0 finished with value: 5.15348184336347 and parameters: {'n_neighbors': 1, 'p': 8}. Best is trial 0 with value: 5.15348184336347.
[I 2024-03-13 18:16:13,237] Trial 1 finished with value: 4.957840751833551 and parameters: {'n_neighbors': 1, 'p': 2}. Best is trial 1 with value: 4.957840751833551.
[I 2024-03-13 18:16:13,604] Trial 2 finished with value: 5.115380215250371 and parameters: {'n_neighbors': 1, 'p': 6}. Best is trial 1 with value: 4.957840751833551.
[I 2024-03-13 18:16:13,992] Trial 3 finished with value: 4.254923914653543 and parameters: {'n_neighbors': 2, 'p': 2}. Best is trial 3 with value: 4.254923914653543.
[I 2024-03-13 18:16:14,361] Trial 4 finished with value: 4.254923914653543 and parameters: {'n_neighbors': 2, 'p': 2}. Best is trial 3 with value: 4.254923914653543.
[I 2024-03-13 18:16:14,786] Trial 5 finished with value: 3

In [ ]:
collagen_knr_resultater_enzym

,Trenings andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,0.2,4.756410,0.348996,3.633472,0.198921,3.094609,0.433005,2.619821,0.113225,"{'n_neighbors': 4, 'p': 8}"
1,0.3,4.750257,0.350679,3.543796,0.193311,3.561828,0.358640,3.181508,0.134417,"{'n_neighbors': 6, 'p': 3}"
2,0.4,4.372122,0.449940,3.134938,0.177458,2.973665,0.434372,2.549921,0.104547,"{'n_neighbors': 9, 'p': 3}"
3,0.5,3.956061,0.549649,2.982222,0.161934,3.356377,0.483322,2.802036,0.118554,"{'n_neighbors': 8, 'p': 2}"
4,0.6,4.575462,0.397586,3.731019,0.202458,3.528880,0.507795,2.773929,0.126463,"{'n_neighbors': 6, 'p': 9}"
5,0.7,4.454108,0.429117,3.617569,0.194244,3.646405,0.425153,2.931505,0.132224,"{'n_neighbors': 8, 'p': 4}"
6,0.8,4.432152,0.434732,3.523944,0.189401,3.869625,0.447834,3.230625,0.150631,"{'n_neighbors': 10, 'p': 7}"
7,0.9,4.411939,0.439876,3.640833,0.193614,3.596400,0.487039,2.981111,0.141053,"{'n_neighbors': 10, 'p': 1}"
8,1.0,4.307204,0.466154,3.551728,0.187885,3.625567,0.481260,3.055603,0.143697,"{'n_neighbors': 9, 'p': 9}"


### **Support Vector Regressor**

In [ ]:
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:
    ubenyttet_andel = (1.0-trenings_andel)
    treningssett_markert, enzymtyper_treningssett_markert,\
    _, _, treningssett_umarkert, _, treningssett\
    = trening_testsett_oppdeling_enzym(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  X_trening = treningssett.iloc[:, :-1]
  X_trening_markert = treningssett_markert.iloc[:, :-1]
  X_trening_umarkert = treningssett_umarkert.iloc[:, :-1]
  y_trening_markert = treningssett_markert.iloc[:, -1]

  # KNR
  def objective(trial):

    parametere = {
          'kernel': trial.suggest_categorical('kernel',["linear", "poly", "rbf"]),
          'C': trial.suggest_int('C', 1.0, 10),
          'degree': trial.suggest_int('degree', 1, 10),
          'gamma': trial.suggest_categorical('gamma', ['scale', 'auto'])

      }

    svr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', SVR(**parametere))
      ])

    splitt = 5
    gjentagelser = 10

    rskf = RepeatedStratifiedKFold(n_splits = splitt,
                                   n_repeats=gjentagelser,
                                   random_state=random_seed)

    rmse_validering_resultat = []

    for train, test in rskf.split(treningssett_markert,
                                  enzymtyper_treningssett_markert):

      X_trening_fold = (treningssett_markert.iloc[train, :]).iloc[:, :-1]
      X_validering_fold = (treningssett_markert.iloc[test, :]).iloc[:, :-1]

      y_trening_fold = (treningssett_markert.iloc[train, :]).iloc[:, -1]
      y_validering_fold = (treningssett_markert.iloc[test, :]).iloc[:, -1]

      svr_pipeline.fit(X_trening_fold, y_trening_fold)

      y_validering_prediksjon = svr_pipeline.predict(X_validering_fold)

      rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
      rmse_validering_resultat.append(rmse_validering)

    rmse_validering_snitt = mean(rmse_validering_resultat)

    return rmse_validering_snitt

  if __name__ == "__main__":
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=10)

  # Bruker beste paramtere for studiet for å lage modell
  beste_parametere = study.best_params

  svr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', SVR(**beste_parametere))
      ])

  # Trener modell for videre evaluering
  svr_pipeline.fit(X_trening_markert, y_trening_markert)
  y_test_prediksjon = knr_pipeline.predict(X_test)
  y_trening_prediksjon = knr_pipeline.predict(X_trening_markert)

  # Henter ut evalueringsresulateter av beste modell fra undersøkelse
  rmse_test, r2_test,\
  mae_test, mape_test,\
  rmse_trening, r2_trening,\
  mae_trening, mape_trening = evaluering_beregning(y_test,
                                                   y_test_prediksjon,
                                                   y_trening_markert,
                                                   y_trening_prediksjon)

  # Legger resulater til i datasett
  resultater = [trenings_andel,
                rmse_test, r2_test, mae_test, mape_test,
                rmse_trening, r2_trening, mae_trening, mape_trening,
                beste_parametere]

  collagen_svr_resultater_enzym.loc[len(collagen_svr_resultater_enzym)] = resultater

[I 2024-03-13 18:17:34,978] A new study created in memory with name: no-name-53f85498-fc78-490c-9fc5-304b74636e31
[I 2024-03-13 18:17:35,619] Trial 0 finished with value: 3.7904467345636945 and parameters: {'kernel': 'rbf', 'C': 5, 'degree': 1, 'gamma': 'auto'}. Best is trial 0 with value: 3.7904467345636945.
[I 2024-03-13 18:17:36,239] Trial 1 finished with value: 3.62551513007233 and parameters: {'kernel': 'rbf', 'C': 8, 'degree': 3, 'gamma': 'auto'}. Best is trial 1 with value: 3.62551513007233.
[I 2024-03-13 18:17:36,861] Trial 2 finished with value: 3.7904467345636945 and parameters: {'kernel': 'rbf', 'C': 5, 'degree': 7, 'gamma': 'auto'}. Best is trial 1 with value: 3.62551513007233.
[I 2024-03-13 18:17:37,644] Trial 3 finished with value: 4.432757379200256 and parameters: {'kernel': 'linear', 'C': 5, 'degree': 1, 'gamma': 'auto'}. Best is trial 1 with value: 3.62551513007233.
[I 2024-03-13 18:17:38,831] Trial 4 finished with value: 3.670453529024988 and parameters: {'kernel': 'r

In [ ]:
collagen_svr_resultater_enzym

,Trenings andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,0.2,4.307204,0.466154,3.551728,0.187885,3.385852,0.321260,2.945079,0.123292,"{'kernel': 'rbf', 'C': 8, 'degree': 3, 'gamma'..."
1,0.3,4.307204,0.466154,3.551728,0.187885,3.842720,0.253494,3.376349,0.138902,"{'kernel': 'rbf', 'C': 7, 'degree': 3, 'gamma'..."
2,0.4,4.307204,0.466154,3.551728,0.187885,3.185608,0.350869,2.676111,0.109010,"{'kernel': 'rbf', 'C': 5, 'degree': 10, 'gamma..."
3,0.5,4.307204,0.466154,3.551728,0.187885,3.713764,0.367432,3.167238,0.133206,"{'kernel': 'poly', 'C': 4, 'degree': 1, 'gamma..."
4,0.6,4.307204,0.466154,3.551728,0.187885,3.700362,0.458797,3.038148,0.138249,"{'kernel': 'linear', 'C': 1, 'degree': 1, 'gam..."
5,0.7,4.307204,0.466154,3.551728,0.187885,3.601753,0.439145,2.996644,0.132832,"{'kernel': 'poly', 'C': 10, 'degree': 1, 'gamm..."
6,0.8,4.307204,0.466154,3.551728,0.187885,3.663763,0.505021,3.036032,0.144439,"{'kernel': 'poly', 'C': 5, 'degree': 1, 'gamma..."
7,0.9,4.307204,0.466154,3.551728,0.187885,3.558889,0.497684,2.958536,0.136675,"{'kernel': 'rbf', 'C': 3, 'degree': 5, 'gamma'..."
8,1.0,4.307204,0.466154,3.551728,0.187885,3.625567,0.481260,3.055603,0.143697,"{'kernel': 'rbf', 'C': 10, 'degree': 3, 'gamma..."


## Alternativ 2: Flere treningssandeler

In [ ]:
collagen_rfr_resultater_dag = pd.DataFrame(columns = ["Trenings andel",
                                                        "RMSE test",
                                                        "R2 test",
                                                        "MAE test",
                                                        "MAPE test",
                                                        "RMSE trening",
                                                        "R2 trening",
                                                        "MAE trening",
                                                        "MAPE trening",
                                                        "Beste parametere"])

collagen_knr_resultater_dag = pd.DataFrame(columns = ["Trenings andel",
                                                         "RMSE test",
                                                         "R2 test",
                                                         "MAE test",
                                                         "MAPE test",
                                                         "RMSE trening",
                                                         "R2 trening",
                                                         "MAE trening",
                                                         "MAPE trening",
                                                         "Beste parametere"])

collagen_svr_resultater_dag = pd.DataFrame(columns = ["Trenings andel",
                                                         "RMSE test",
                                                         "R2 test",
                                                         "MAE test",
                                                         "MAPE test",
                                                         "RMSE trening",
                                                         "R2 trening",
                                                         "MAE trening",
                                                         "MAPE trening",
                                                         "Beste parametere"])



test_andel = 0.2
trening_markert, test,\
trening_umarkert, trening = trening_testsett_oppdeling_dag(rå_data,
                                                          collagen_data_design_uten_NIR,
                                                          test_andel=test_andel,
                                                          trening_krav=True)

X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]

# Treningsandeler
trenings_andeler = [0.8, 0.9, 1.0]

### **RandomForestRegressor**

In [ ]:
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  X_trening = treningssett.iloc[:, :-1]
  X_trening_markert = treningssett_markert.iloc[:, :-1]
  X_trening_umarkert = treningssett_umarkert.iloc[:, :-1]
  y_trening_markert = treningssett_markert.iloc[:, -1]

  # RandomForestRegressor
  def objective(trial):

    parametere = {
          'n_estimators': trial.suggest_int('n_estimators', 100, 200),
          'max_depth': trial.suggest_int('max_depth', 1, 10)
      }
    rfr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', RandomForestRegressor(**parametere,
                                           random_state=random_seed))
      ])

    splitt = 2

    treningssett, valideringssett = trening_validering_oppdeling_dag(rå_data,
                                                                     treningssett_markert,
                                                                     n_splitt=splitt,
                                                                     trening_krav=True)
    rmse_validering_resultat = []

    for i, j in zip(treningssett, valideringssett):
      train = treningssett[i]
      val = valideringssett[j]

      X_trening_fold = (trening_markert.loc[train, :]).iloc[:, :-1]
      X_validering_fold = (trening_markert.loc[val, :]).iloc[:, :-1]

      y_trening_fold = (trening_markert.loc[train, :]).iloc[:, -1]
      y_validering_fold = (trening_markert.loc[val, :]).iloc[:, -1]

      rfr_pipeline.fit(X_trening_fold, y_trening_fold)

      y_validering_prediksjon = rfr_pipeline.predict(X_validering_fold)
      rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
      rmse_validering_resultat.append(rmse_validering)

    rmse_validering_snitt = mean(rmse_validering_resultat)

    return rmse_validering_snitt


  if __name__ == "__main__":
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=100)


  # Bruker beste paramtere for studiet for å lage modell
  beste_parametere = study.best_params

  rfr_pipeline = Pipeline([
      ('skalerer', StandardScaler()),
      ('modell', RandomForestRegressor(**beste_parametere,
                                       random_state=random_seed))
  ])

  # Trener modell for videre evaluering
  rfr_pipeline.fit(X_trening_markert, y_trening_markert)
  y_test_prediksjon = rfr_pipeline.predict(X_test)
  y_trening_prediksjon = rfr_pipeline.predict(X_trening_markert)

  # Henter ut evalueringsresulateter av beste modell fra undersøkelse
  rmse_test, r2_test, mae_test, mape_test,\
  rmse_trening, r2_trening, mae_trening, mape_trening\
  = evaluering_beregning(y_test,
                         y_test_prediksjon,
                         y_trening_markert,
                         y_trening_prediksjon)


  # Legger resulater til i datasett
  resultater = [trenings_andel,
                rmse_test, r2_test, mae_test, mape_test,
                rmse_trening, r2_trening, mae_trening, mape_trening,
                beste_parametere]

  collagen_rfr_resultater_dag.loc[len(collagen_rfr_resultater_dag)] = resultater

[I 2024-03-13 23:00:12,138] A new study created in memory with name: no-name-a01ed145-38a3-4d09-b37a-676683fa9876
[I 2024-03-13 23:00:13,646] Trial 0 finished with value: 3.796984304282677 and parameters: {'n_estimators': 184, 'max_depth': 6}. Best is trial 0 with value: 3.796984304282677.
[I 2024-03-13 23:00:14,991] Trial 1 finished with value: 3.753802900291549 and parameters: {'n_estimators': 168, 'max_depth': 7}. Best is trial 1 with value: 3.753802900291549.
[I 2024-03-13 23:00:16,184] Trial 2 finished with value: 3.774555885235837 and parameters: {'n_estimators': 149, 'max_depth': 6}. Best is trial 1 with value: 3.753802900291549.
[I 2024-03-13 23:00:17,481] Trial 3 finished with value: 4.268282413840528 and parameters: {'n_estimators': 148, 'max_depth': 2}. Best is trial 1 with value: 3.753802900291549.
[I 2024-03-13 23:00:18,545] Trial 4 finished with value: 5.023118309586517 and parameters: {'n_estimators': 131, 'max_depth': 1}. Best is trial 1 with value: 3.753802900291549.
[

KeyboardInterrupt: 

In [ ]:
collagen_rfr_resultater_dag

### **KNeighborRegressor**

In [ ]:
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  X_trening = treningssett.iloc[:, :-1]
  X_trening_markert = treningssett_markert.iloc[:, :-1]
  X_trening_umarkert = treningssett_umarkert.iloc[:, :-1]
  y_trening_markert = treningssett_markert.iloc[:, -1]

  # KneigborsRegressor
  def objective(trial):

    parametere = {
          'n_neighbors': trial.suggest_int('n_neighbors', 1, 20),
          'p': trial.suggest_int('p', 1, 20)
      }
    knr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', KNeighborsRegressor(**parametere))
      ])


    splitt = 2

    treningssett, valideringssett = trening_validering_oppdeling_dag(rå_data,
                                                                      treningssett_markert,
                                                                      n_splitt=splitt,
                                                                      trening_krav=True)
    rmse_validering_resultat = []

    for i, j in zip(treningssett, valideringssett):
      train = treningssett[i]
      val = valideringssett[j]

      X_trening_fold = (trening_markert.loc[train, :]).iloc[:, :-1]
      X_validering_fold = (trening_markert.loc[val, :]).iloc[:, :-1]

      y_trening_fold = (trening_markert.loc[train, :]).iloc[:, -1]
      y_validering_fold = (trening_markert.loc[val, :]).iloc[:, -1]

      knr_pipeline.fit(X_trening_fold, y_trening_fold)

      y_validering_prediksjon = knr_pipeline.predict(X_validering_fold)
      rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
      rmse_validering_resultat.append(rmse_validering)

    rmse_validering_snitt = mean(rmse_validering_resultat)

    return rmse_validering_snitt


  if __name__ == "__main__":
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=100)


  # Bruker beste paramtere for studiet for å lage modell
  beste_parametere = study.best_params

  knr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', KNeighborsRegressor(**beste_parametere))
      ])

  # Trener modell for videre evaluering
  knr_pipeline.fit(X_trening_markert, y_trening_markert)
  y_test_prediksjon = knr_pipeline.predict(X_test)
  y_trening_prediksjon = knr_pipeline.predict(X_trening_markert)

  # Henter ut evalueringsresulateter av beste modell fra undersøkelse
  rmse_test, r2_test,\
  mae_test, mape_test,\
  rmse_trening, r2_trening,\
  mae_trening, mape_trening = evaluering_beregning(y_test,
                                                   y_test_prediksjon,
                                                   y_trening_markert,
                                                   y_trening_prediksjon)

  # Legger resulater til i datasett
  resultater = [trenings_andel,
                rmse_test, r2_test, mae_test, mape_test,
                rmse_trening, r2_trening, mae_trening, mape_trening,
                beste_parametere]

  collagen_knr_resultater_dag.loc[len(collagen_knr_resultater_dag)] = resultater

[I 2024-03-13 23:00:35,673] A new study created in memory with name: no-name-5da69154-224d-4077-ae0b-05c4196416ec
[I 2024-03-13 23:00:36,500] Trial 0 finished with value: 5.004266801871374 and parameters: {'n_neighbors': 14, 'p': 15}. Best is trial 0 with value: 5.004266801871374.
[I 2024-03-13 23:00:38,278] Trial 1 finished with value: 5.157710462165955 and parameters: {'n_neighbors': 19, 'p': 5}. Best is trial 0 with value: 5.004266801871374.
[I 2024-03-13 23:00:39,495] Trial 2 finished with value: 4.5030614348004 and parameters: {'n_neighbors': 11, 'p': 1}. Best is trial 2 with value: 4.5030614348004.
[I 2024-03-13 23:00:40,943] Trial 3 finished with value: 4.688114562376997 and parameters: {'n_neighbors': 11, 'p': 13}. Best is trial 2 with value: 4.5030614348004.
[I 2024-03-13 23:00:41,890] Trial 4 finished with value: 4.5859244572412505 and parameters: {'n_neighbors': 8, 'p': 17}. Best is trial 2 with value: 4.5030614348004.
[I 2024-03-13 23:00:42,998] Trial 5 finished with value:

KeyboardInterrupt: 

In [ ]:
collagen_knr_resultater_dag

### **Support Vector Regressor**

In [ ]:
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  X_trening = treningssett.iloc[:, :-1]
  X_trening_markert = treningssett_markert.iloc[:, :-1]
  X_trening_umarkert = treningssett_umarkert.iloc[:, :-1]
  y_trening_markert = treningssett_markert.iloc[:, -1]

  # SupportVectorRegressor
  def objective(trial):

    parametere = {
          'kernel': trial.suggest_categorical('kernel',["linear", "poly", "rbf"]),
          'C': trial.suggest_categorical('C', [0.1, 1, 10]),
          'degree': trial.suggest_int('degree', 1, 10)
      }

    svr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', SVR(**parametere))
      ])


    splitt = 2

    treningssett, valideringssett = trening_validering_oppdeling_dag(rå_data,
                                                                      treningssett_markert,
                                                                      n_splitt=splitt,
                                                                      trening_krav=True)
    rmse_validering_resultat = []

    for i, j in zip(treningssett, valideringssett):
      train = treningssett[i]
      val = valideringssett[j]

      X_trening_fold = (trening_markert.loc[train, :]).iloc[:, :-1]
      X_validering_fold = (trening_markert.loc[val, :]).iloc[:, :-1]

      y_trening_fold = (trening_markert.loc[train, :]).iloc[:, -1]
      y_validering_fold = (trening_markert.loc[val, :]).iloc[:, -1]

      svr_pipeline.fit(X_trening_fold, y_trening_fold)

      y_validering_prediksjon = svr_pipeline.predict(X_validering_fold)
      rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
      rmse_validering_resultat.append(rmse_validering)

    rmse_validering_snitt = mean(rmse_validering_resultat)

    return rmse_validering_snitt


  if __name__ == "__main__":
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=100)


  # Bruker beste paramtere for studiet for å lage modell
  beste_parametere = study.best_params

  svr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', SVR(**beste_parametere))
      ])

  # Trener modell for videre evaluering
  svr_pipeline.fit(X_trening_markert, y_trening_markert)
  y_test_prediksjon = svr_pipeline.predict(X_test)
  y_trening_prediksjon = svr_pipeline.predict(X_trening_markert)

  # Henter ut evalueringsresulateter av beste modell fra undersøkelse
  rmse_test, r2_test,\
  mae_test, mape_test,\
  rmse_trening, r2_trening,\
  mae_trening, mape_trening = evaluering_beregning(y_test,
                                                   y_test_prediksjon,
                                                   y_trening_markert,
                                                   y_trening_prediksjon)

  # Legger resulater til i datasett
  resultater = [trenings_andel,
                rmse_test, r2_test, mae_test, mape_test,
                rmse_trening, r2_trening, mae_trening, mape_trening,
                beste_parametere]

  collagen_svr_resultater_dag.loc[len(collagen_svr_resultater_dag)] = resultater

[I 2024-03-13 23:01:11,665] A new study created in memory with name: no-name-e979a9f0-8f1c-422b-affd-ab69514c3002
[I 2024-03-13 23:01:12,270] Trial 0 finished with value: 4.546318014245733 and parameters: {'kernel': 'linear', 'C': 0.1, 'degree': 10}. Best is trial 0 with value: 4.546318014245733.
[I 2024-03-13 23:01:12,818] Trial 1 finished with value: 9.297602010201523 and parameters: {'kernel': 'linear', 'C': 10, 'degree': 9}. Best is trial 0 with value: 4.546318014245733.
[I 2024-03-13 23:01:13,385] Trial 2 finished with value: 9.297602010201523 and parameters: {'kernel': 'linear', 'C': 10, 'degree': 3}. Best is trial 0 with value: 4.546318014245733.
[I 2024-03-13 23:01:14,013] Trial 3 finished with value: 9.818960714969776 and parameters: {'kernel': 'linear', 'C': 1, 'degree': 9}. Best is trial 0 with value: 4.546318014245733.
[I 2024-03-13 23:01:14,738] Trial 4 finished with value: 5.774523272878266 and parameters: {'kernel': 'rbf', 'C': 0.1, 'degree': 8}. Best is trial 0 with val

KeyboardInterrupt: 

In [ ]:
collagen_svr_resultater_dag